In [2]:
# =============================================================================
# 1. IMPORT AND INSTALL DEPENDENCIES
# =============================================================================
# If needed, install dependencies using pip (uncomment the following lines):
# !pip install opencv-python mediapipe torch torchvision scikit-learn

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [2]:
#!pip install transformers

In [2]:
#!pip install mediapipe

In [3]:
#!pip install scikit-learn

In [3]:
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report

In [1]:
# Sample all videos with uniform frames (Select 60 frames out of entire video)

import os
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe modules.
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    """
    Extract keypoint values from MediaPipe detection results.
    Returns a concatenated numpy array of pose, face, left hand, and right hand keypoints.
    """
    # Pose (33 landmarks x [x, y, z, visibility])
    pose = np.array([[res.x, res.y, res.z, res.visibility] 
                     for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    # Face (468 landmarks x [x, y, z])
    face = np.array([[res.x, res.y, res.z] 
                     for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    # Left hand (21 landmarks x [x, y, z])
    lh = np.array([[res.x, res.y, res.z] 
                   for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    # Right hand (21 landmarks x [x, y, z])
    rh = np.array([[res.x, res.y, res.z] 
                   for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

def sample_frames_uniformly(frames, target_count=60):
    """
    Given a list of frames, return exactly 'target_count' frames by uniformly sampling (or repeating).
    """
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        # Downsample: select 60 frames uniformly
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        # Upsample: repeat frames to reach target_count
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    """
    Reads a video, uniformly samples exactly 'target_frames' frames,
    processes each sampled frame with MediaPipe Holistic to extract keypoints,
    and saves each frame's keypoints as a separate .npy file in 'output_folder'.
    """
    # Create output folder if it doesn't exist.
    os.makedirs(output_folder, exist_ok=True)
    
    # 1. Read all frames from the video.
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    # 2. Uniformly sample frames to get exactly 'target_frames' frames.
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    # 3. Process each sampled frame and save keypoints.
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            # Extract keypoints
            keypoints = extract_all_keypoints(results)
            # Save each frame's keypoints as a separate .npy file.
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# -----------------------------------------------------------------------------
# MAIN EXECUTION
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Define your classes. For example, these folders are expected under './dataset'.
    actions = actions = [
    "Death",
    "Doctor",
    "he",
    "healthy",
    "Hospital",
    "I",
    "Medicine",
    "Patient",
    "she",
    "sick"
    ]


    dataset_dir = "./dataset"  # This is where your video files are stored, one folder per class.
    
    # Define a separate output folder for extracted keypoints to avoid clutter.
    output_root = "./extracted_keypoints"
    os.makedirs(output_root, exist_ok=True)
    
    # Process each class folder.
    for action in actions:
        class_folder = os.path.join(dataset_dir, action)
        # Create a corresponding output folder for the class.
        output_class_folder = os.path.join(output_root, action)
        os.makedirs(output_class_folder, exist_ok=True)
        
        # List all video files in the class folder.
        video_files = [vf for vf in os.listdir(class_folder) 
                       if vf.lower().endswith((".mp4", ".avi", ".mov"))]
        
        # Process each video file.
        for video_file in video_files:
            video_path = os.path.join(class_folder, video_file)
            # Create an output subfolder for this video to store its 60 .npy files.
            base_name = os.path.splitext(video_file)[0]
            video_output_folder = os.path.join(output_class_folder, base_name)
            os.makedirs(video_output_folder, exist_ok=True)
            
            print(f"Processing video: {video_path}")
            process_video_and_save_frames(video_path, video_output_folder, target_frames=60)
    
    print("All videos processed: 60 keypoint .npy files per video have been saved.")


Processing video: ./dataset/Death/MVI_8932.MP4


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Saved ./extracted_keypoints/Death/MVI_8932/frame_0.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_1.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_2.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_3.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_4.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_5.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_6.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_7.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_8.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_9.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_10.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_11.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_12.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/MVI_8932/frame_13.npy, shape: (1662,)
Sa

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_6.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_7.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_8.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_9.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_10.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_11.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_12.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_13.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_14.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_15.npy, shape: (1662,)
Saved ./extracted_keypoints/Death/Death11 - Made with Clipchamp/frame_16.n

...

Saved ./extracted_keypoints/sick/MVI_9282/frame_0.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_1.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_2.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_3.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_4.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_5.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_6.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_7.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_8.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_9.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_10.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_11.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_12.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/MVI_9282/frame_13.npy, shape: (1662,)
Saved ./extracte

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Saved ./extracted_keypoints/sick/sick_9/frame_0.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_1.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_2.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_3.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_4.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_5.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_6.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_7.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_8.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_9.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_10.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_11.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_12.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/frame_13.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_9/fram

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Saved ./extracted_keypoints/sick/sick_10/frame_0.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_1.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_2.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_3.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_4.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_5.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_6.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_7.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_8.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_9.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_10.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_11.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_12.npy, shape: (1662,)
Saved ./extracted_keypoints/sick/sick_10/frame_13.npy, shape: (1662,)
Saved ./extracted_keypoints/si

In [6]:
#Checking to see if the extracted numpy arrays work

import numpy as np

# Specify the file path to your .npy file.
file_path = "./extracted_keypoints/he/MVI_9891/frame_57.npy"

# Load the numpy array from the file.
keypoints = np.load(file_path)

# Optionally, print the shape and some values to verify.
print("Shape of loaded keypoints:", keypoints.shape)
print("Keypoints data:", keypoints)


Shape of loaded keypoints: (1662,)
Keypoints data: [ 0.48063502  0.25733462 -0.54641551 ...  0.3982769   0.83270705
  0.00572504]


In [5]:
#PreProcessing the data from extracted keypoints

import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# -----------------------------
# Custom to_categorical function (no TensorFlow dependency)
# -----------------------------
def to_categorical(y, num_classes):
    y = np.array(y, dtype=int)
    return np.eye(num_classes)[y]

# -----------------------------
# Dataset Class – Modified to use exactly 14 videos per class
# -----------------------------
class SignLanguageVideoDataset(Dataset):
    """
    Custom Dataset for loading keypoint sequences extracted from videos.
    Assumes the data is organized as:
      ./extracted_keypoints/<action>/<video_basename>/frame_*.npy
    For each action/class, this version randomly samples exactly num_videos_per_class
    valid video folders (if available). A valid video folder is one that has exactly
    the required number of frames (sequence_length).
    """
    def __init__(self, data_dir, actions, sequence_length=60, num_videos_per_class=14, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment

        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue

            # List all subfolders (video folders) for this action
            video_folders = [vf for vf in os.listdir(action_folder)
                             if os.path.isdir(os.path.join(action_folder, vf))]
            valid_video_folders = []
            for video_folder in video_folders:
                video_path = os.path.join(action_folder, video_folder)
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                # Sort files by numeric index (assuming naming like "frame_0.npy", etc.)
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    valid_video_folders.append(video_folder)
            
            # If there are more than the required videos, randomly sample the required number.
            if len(valid_video_folders) >= num_videos_per_class:
                sampled_folders = random.sample(valid_video_folders, num_videos_per_class)
            else:
                # If fewer are available, use all (or you might decide to skip this class).
                sampled_folders = valid_video_folders

            for video_folder in sampled_folders:
                video_path = os.path.join(action_folder, video_folder)
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                frames = []
                for f in frame_files:
                    fpath = os.path.join(video_path, f)
                    frames.append(np.load(fpath))
                # Stack the frames to create an array of shape (sequence_length, keypoint_dim)
                sequence = np.stack(frames, axis=0)
                self.data.append(sequence)
                self.labels.append(label)
        
        # Convert lists to numpy arrays.
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sample = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# -----------------------------
# Main Preprocessing and Split
# -----------------------------
if __name__ == "__main__":
    # Root folder where keypoints have been extracted.
    data_dir = "./extracted_keypoints"
    actions = actions = [
    "Death",
    "Doctor",
    "he",
    "healthy",
    "Hospital",
    "I",
    "Medicine",
    "Patient",
    "she",
    "sick"
    ]
    
    # Create the dataset with exactly 14 videos per class (if available)
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length=60, num_videos_per_class=14)
    print("Data shape:", dataset.data.shape)    # e.g., (number_of_classes * 14, 60, keypoint_dim)
    print("Labels shape:", dataset.labels.shape)
    
    num_classes = len(actions)
    categorical_labels = to_categorical(dataset.labels, num_classes=num_classes)
    print("Categorical labels shape:", categorical_labels.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset.data, categorical_labels,
                                                        test_size=0.2, random_state=42)
    print("Train data shape:", X_train.shape)
    print("Test data shape:", X_test.shape)
    
    batch_size = 8
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                     torch.tensor(y_train, dtype=torch.float32))
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                    torch.tensor(y_test, dtype=torch.float32))
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    print("Preprocessing complete. Ready to train your model.")


Data shape: (490, 60, 1662)
Labels shape: (490,)
Categorical labels shape: (490, 35)
Train data shape: (392, 60, 1662)
Test data shape: (98, 60, 1662)
Preprocessing complete. Ready to train your model.


In [6]:
#Checking model parameters

for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.size()}")
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters:", total_params)


NameError: name 'model' is not defined

In [11]:
# 1. IMPORT AND INSTALL DEPENDENCIES
# =============================================================================
import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


In [13]:
import os
import random
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# -----------------------------
# Custom to_categorical function (for verification, not used for CrossEntropyLoss)
# -----------------------------
def to_categorical(y, num_classes):
    y = np.array(y, dtype=int)
    return np.eye(num_classes)[y]

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            # Use landmarks 11 and 12 as shoulders
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        norm_pose = np.hstack([(pose_points[:, 0:2] - center) / scale,
                                pose_points[:, 2:3],
                                pose_points[:, 3:4]])
    else:
        norm_pose = np.zeros((33, 4))
    
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([(face_points[:, 0:2] - center) / scale,
                                face_points[:, 2:3]])
    else:
        norm_face = np.zeros((468, 3))
    
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]  # assume index 0 is the wrist
        norm_left_hand = np.hstack([(left_hand_points[:, 0:2] - wrist) / scale,
                                     left_hand_points[:, 2:3]])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([(right_hand_points[:, 0:2] - wrist) / scale,
                                      right_hand_points[:, 2:3]])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    """
    Process a video: uniformly sample 'target_frames', extract normalized keypoints,
    and save each frame's keypoints as separate .npy files.
    """
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS – Modified to Use Exactly 14 Videos Per Class
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    """
    Custom dataset for loading keypoint sequences.
    Assumes the data is organized as:
      ./extracted_keypoints/<action>/<video_basename>/frame_*.npy
    For each action, randomly sample exactly 'num_videos_per_class' valid video folders (if available).
    """
    def __init__(self, data_dir, actions, sequence_length=60, num_videos_per_class=14, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            # List all subfolders (video folders) for this action
            video_folders = [vf for vf in os.listdir(action_folder) if os.path.isdir(os.path.join(action_folder, vf))]
            valid_video_folders = []
            for vf in video_folders:
                video_path = os.path.join(action_folder, vf)
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    valid_video_folders.append(vf)
            # Sample exactly num_videos_per_class if possible
            if len(valid_video_folders) >= num_videos_per_class:
                sampled_folders = random.sample(valid_video_folders, num_videos_per_class)
            else:
                sampled_folders = valid_video_folders
            for vf in sampled_folders:
                video_path = os.path.join(action_folder, vf)
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                frames = []
                for f in frame_files:
                    fpath = os.path.join(video_path, f)
                    frames.append(np.load(fpath))
                sequence = np.stack(frames, axis=0)  # Shape: (sequence_length, keypoint_dim)
                self.data.append(sequence)
                self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.augment:
            sample = augment_sequence(sample)
        sample = torch.tensor(sample, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    """
    A 3-layer LSTM model with dropout and two dense layers.
    """
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = out[:, -1, :]  # Use the last time step
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length,
                                       num_videos_per_class=14, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    sample, _ = dataset[0]
    input_size = sample.shape[1]
    num_classes = len(actions)
    
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        elapsed = time.time() - start_time
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    
    torch.save(model.state_dict(), "action_detection_model60_2.pth")
    print("Model weights saved as 'action_detection_model60_2.pth'")
    return model

# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length,
                                       num_videos_per_class=14, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# MAIN PREPROCESSING AND TRAINING SPLIT (for verification)
# =============================================================================
if __name__ == "__main__":
    data_dir = "./extracted_keypoints"
    actions = [
        "I","Medicine","Price","Sick"
    ]
    
    # Create the dataset with exactly 14 videos per class (if available)
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length=60, num_videos_per_class=14)
    print("Data shape:", dataset.data.shape)    # Expected: (number_of_classes * 14, 60, keypoint_dim)
    print("Labels shape:", dataset.labels.shape)
    
    num_classes = len(actions)
    categorical_labels = to_categorical(dataset.labels, num_classes=num_classes)
    print("Categorical labels shape:", categorical_labels.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset.data, categorical_labels,
                                                        test_size=0.2, random_state=42)
    print("Train data shape:", X_train.shape)
    print("Test data shape:", X_test.shape)
    
    batch_size = 8
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                    torch.tensor(y_train, dtype=torch.float32))
    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                   torch.tensor(y_test, dtype=torch.float32))
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    print("Preprocessing complete. Ready to train your model.")
    
    # Train the model using extracted keypoints (14 videos per class)
    model = train_model(data_dir, actions, sequence_length=60, batch_size=32,
                        num_epochs=300, learning_rate=0.001, augment=True)
    
    # Evaluate the trained model on test data
    evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32)


Data shape: (56, 60, 1662)
Labels shape: (56,)
Categorical labels shape: (56, 4)
Train data shape: (44, 60, 1662)
Test data shape: (12, 60, 1662)
Preprocessing complete. Ready to train your model.
Epoch [1/300], Loss: 1.3889, Time: 0.10s
Epoch [2/300], Loss: 1.3865, Time: 0.11s
Epoch [3/300], Loss: 1.3890, Time: 0.10s
Epoch [4/300], Loss: 1.3853, Time: 0.10s
Epoch [5/300], Loss: 1.3874, Time: 0.10s
Epoch [6/300], Loss: 1.3880, Time: 0.10s
Epoch [7/300], Loss: 1.3879, Time: 0.11s
Epoch [8/300], Loss: 1.3844, Time: 0.10s
Epoch [9/300], Loss: 1.3854, Time: 0.11s
Epoch [10/300], Loss: 1.3855, Time: 0.11s
Epoch [11/300], Loss: 1.3856, Time: 0.11s
Epoch [12/300], Loss: 1.3831, Time: 0.11s
Epoch [13/300], Loss: 1.3802, Time: 0.10s
Epoch [14/300], Loss: 1.3707, Time: 0.11s
Epoch [15/300], Loss: 1.3585, Time: 0.10s
Epoch [16/300], Loss: 1.3565, Time: 0.11s
Epoch [17/300], Loss: 1.2670, Time: 0.11s
Epoch [18/300], Loss: 1.3813, Time: 0.11s
Epoch [19/300], Loss: 1.5389, Time: 0.10s
Epoch [20/300]

/home/goku/anaconda3/envs/comfyui/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/goku/anaconda3/envs/comfyui/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/goku/anaconda3/envs/comfyui/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [1]:
#Previous TestML code training only

# =============================================================================
# 1. IMPORT AND INSTALL DEPENDENCIES
# =============================================================================
import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import random


# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    """
    Extract and normalize keypoints from MediaPipe detection results.
    Normalization: Uses the midpoint of the left and right shoulders (if available)
    as the center, and the shoulder distance as the scale factor.
    Returns a concatenated numpy array of normalized pose, face, left hand, and right hand keypoints.
    """
    # Normalize pose keypoints
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            # Landmarks 11 and 12 correspond to left and right shoulders
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        # Only normalize x and y coordinates; leave z and visibility unchanged
        norm_pose = np.hstack([ (pose_points[:,0:2] - center) / scale, pose_points[:,2:3], pose_points[:,3:4] ])
    else:
        norm_pose = np.zeros((33, 4))
    
    # Normalize face keypoints using the same center and scale
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face_points[:,0:2] - center) / scale, face_points[:,2:3] ])
    else:
        norm_face = np.zeros((468, 3))
    
    # Normalize left hand keypoints (using the wrist as the anchor)
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]  # assume index 0 is the wrist
        norm_left_hand = np.hstack([ (left_hand_points[:,0:2] - wrist) / scale, left_hand_points[:,2:3] ])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    # Normalize right hand keypoints (using the wrist as the anchor)
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([ (right_hand_points[:,0:2] - wrist) / scale, right_hand_points[:,2:3] ])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    """
    Apply augmentation to a keypoint sequence by adding Gaussian noise.
    Input:
      sequence: numpy array of shape (sequence_length, keypoint_dim)
    Returns an augmented sequence.
    """
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    """
    Given a list of frames, return exactly 'target_count' frames by uniformly sampling.
    """
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    """
    Process a video: sample frames, extract normalized keypoints using MediaPipe Holistic,
    and save each frame's keypoints as a separate .npy file.
    """
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS WITH AUGMENTATION OPTION
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    """
    Custom dataset for loading keypoint sequences.
    Expects data organized as:
      ./extracted_keypoints/<action>/<video_basename>/frame_*.npy
    """
    def __init__(self, data_dir, actions, sequence_length=60, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            for video_folder in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_folder)
                if not os.path.isdir(video_path):
                    continue
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    frames = []
                    for f in frame_files:
                        fpath = os.path.join(video_path, f)
                        frames.append(np.load(fpath))
                    sequence = np.stack(frames, axis=0)  # (sequence_length, keypoint_dim)
                    self.data.append(sequence)
                    self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if self.augment:
            sequence = augment_sequence(sequence)
        sample = torch.tensor(sequence, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    """
    3-layer LSTM architecture with dropout regularization and two dense layers.
    Architecture:
      LSTM(64) -> ReLU -> Dropout ->
      LSTM(128) -> ReLU -> Dropout ->
      LSTM(256) -> ReLU -> Dropout ->
      Dense(64) -> ReLU -> Dropout ->
      Dense(num_classes)
    """
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        # Use the output from the last time step
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    """
    Train the LSTM model. Optionally applies data augmentation.
    Early-stop once train acc ≥ 90%.
    """
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    sample, _   = dataset[0]
    input_size  = sample.shape[1]
    num_classes = len(actions)

    model     = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    for epoch in range(1, num_epochs+1):
        model.train()
        running_loss = 0.0
        correct = total = 0

        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels    = labels.to(device)

            optimizer.zero_grad()
            outputs = model(sequences)
            loss    = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

        train_acc = 100 * correct / total
        avg_loss  = running_loss / len(dataloader)
        print(f"Epoch [{epoch}/{num_epochs}] — Loss: {avg_loss:.4f} — Acc: {train_acc:.2f}%")

        # Early stopping condition
        if train_acc >= 95.0:
            print(f"→ Early stopping at epoch {epoch} (train acc {train_acc:.2f}%)")
            break

    torch.save(model.state_dict(), "action_detection_model60_29.pth")
    print("Model weights saved as 'action_detection_model60_29.pth'")
    return model


# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# 9. REAL-TIME SLIDING WINDOW PREDICTION
# =============================================================================
def sliding_window_prediction(video_file, model, actions, window_size=60, smoothing_window=10, threshold=0.5):
    """
    Processes a video file using a sliding window for real-time prediction.
    Applies a smoothing mechanism over recent predictions.
    """
    colors = [(245,117,16), (117,245,16), (16,117,245), (0,255,0)]
    sequence = []
    predictions = []
    sentence = []
    
    cap = cv2.VideoCapture(video_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            keypoints = extract_all_keypoints(results)
            sequence.append(keypoints)
            if len(sequence) > window_size:
                sequence = sequence[-window_size:]
            
            if len(sequence) == window_size:
                seq_array = np.stack(sequence, axis=0)
                seq_tensor = torch.tensor(seq_array, dtype=torch.float32).unsqueeze(0).to(device)
                model.eval()
                with torch.no_grad():
                    outputs = model(seq_tensor)
                    prob = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                    pred_idx = np.argmax(prob)
                    predictions.append(pred_idx)
                
                if len(predictions) >= smoothing_window:
                    recent_preds = predictions[-smoothing_window:]
                    if np.unique(recent_preds).size == 1 and prob[pred_idx] > threshold:
                        if len(sentence) == 0 or actions[pred_idx] != sentence[-1]:
                            sentence.append(actions[pred_idx])
                
                vis_frame = prob_viz(prob, actions, image, colors)
                cv2.rectangle(vis_frame, (0,0), (640,40), (245,117,16), -1)
                cv2.putText(vis_frame, ' '.join(sentence), (3,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Sliding Window Action Detection", vis_frame)
            else:
                cv2.imshow("Sliding Window Action Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# =============================================================================
# 10. PROBABILITY VISUALIZATION FUNCTION
# =============================================================================
def prob_viz(probabilities, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(probabilities):
        width = int(prob * 200)
        top_left = (0, 60 + num * 40)
        bottom_right = (width, 90 + num * 40)
        cv2.rectangle(output_frame, top_left, bottom_right, colors[num], -1)
        cv2.putText(output_frame, f"{actions[num]}: {prob:.2f}", (10, 85 + num * 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

# =============================================================================
# 11. DEVICE CONFIGURATION AND MAIN EXECUTION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if __name__ == "__main__":

    # Define the action classes
    actions = [
    "Death",
    "Doctor",
    "he",
    "healthy",
    "Hospital",
    "I",
    "Medicine",
    "Patient",
    "she",
    "sick"
    ]
    
    # -----------------------------
    # Step 1: Extract keypoints from raw training videos
    # -----------------------------
    # raw_dataset_dir: folder containing raw video files organized by action
    raw_dataset_dir = "./dataset"  # e.g., ./dataset/I, ./dataset/Medicine, etc.
    # extracted_keypoints_dir: destination folder for saving keypoint files
    extracted_keypoints_dir = "./extracted_keypoints"
    
    # Loop over each action folder in the raw dataset
    '''for action in actions:
        action_raw_folder = os.path.join(raw_dataset_dir, action)
        action_extracted_folder = os.path.join(extracted_keypoints_dir, action)
        os.makedirs(action_extracted_folder, exist_ok=True)
        
        # Get all video files for the action
        video_files = [vf for vf in os.listdir(action_raw_folder) if vf.lower().endswith((".mp4", ".avi", ".mov"))]
        for video_file in video_files:
            video_path = os.path.join(action_raw_folder, video_file)
            video_basename = os.path.splitext(video_file)[0]
            video_output_folder = os.path.join(action_extracted_folder, video_basename)
            os.makedirs(video_output_folder, exist_ok=True)
            print(f"Processing video: {video_path}")
            process_video_and_save_frames(video_path, video_output_folder, target_frames=60)'''
    
    # -----------------------------
    # Step 2: Train the model using extracted keypoints
    # -----------------------------
    model = train_model(extracted_keypoints_dir, actions, sequence_length=60, batch_size=32,
                        num_epochs=16000, learning_rate=0.001, augment=True)
    
    # -----------------------------
    # Step 3: Evaluate the trained model on the test data (pre-extracted keypoints)
    # -----------------------------
    evaluate_model(model, extracted_keypoints_dir, actions, sequence_length=60, batch_size=32)
    
    # -----------------------------
    # Step 4: Evaluate a single raw video (no pre-extracted frames)
    # -----------------------------
    # This function reads the raw video, extracts 60 frames uniformly, computes keypoints,
    # and then feeds the sequence into the model.
    def evaluate_raw_video(model, video_file, sequence_length=60):
        """
        Evaluates a raw video file (without pre-extracted keypoints).
        This function:
          1. Reads the video and uniformly samples exactly 'sequence_length' frames.
          2. Processes each sampled frame with MediaPipe Holistic to extract normalized keypoints.
          3. Stacks the keypoints into a sequence.
          4. Feeds the sequence into the trained model to obtain a prediction.
          
        Returns:
          pred_idx: the predicted class index.
          probabilities: the softmax probabilities for each class.
        """
        # Open the video file.
        cap = cv2.VideoCapture(video_file)
        all_frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            all_frames.append(frame)
        cap.release()
    
        # Uniformly sample the frames.
        sampled_frames = sample_frames_uniformly(all_frames, target_count=sequence_length)
        
        # Process each frame using MediaPipe to extract normalized keypoints.
        keypoint_list = []
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            for frm in sampled_frames:
                image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                keypoints = extract_all_keypoints(results)
                keypoint_list.append(keypoints)
        
        # Stack keypoints to form a sequence.
        sequence = np.stack(keypoint_list, axis=0)  # Shape: (sequence_length, keypoint_dim)
        
        # Convert to tensor and add batch dimension.
        sample_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).to(device)
    
        # Evaluate using the trained model.
        model.eval()
        with torch.no_grad():
            outputs = model(sample_tensor)
            probabilities = torch.softmax(outputs, dim=1).cpu().numpy()[0]
            pred_idx = np.argmax(probabilities)
    
        return pred_idx, probabilities

    # Path to a raw video sample (without pre-extracted frames)
    '''raw_video_file = "./videos/SourikSampleVideo.mp4"
    pred_idx, probabilities = evaluate_raw_video(model, raw_video_file, sequence_length=60)
    if pred_idx is not None:
        print("Raw Video Evaluation:")
        print("Predicted Class:", actions[pred_idx])
        print("Probabilities:", probabilities)
    
    # -----------------------------
    # Step 5: Optionally run real-time sliding window prediction
    # -----------------------------
    test_video_file = raw_video_file  # Adjust path as needed
    sliding_window_prediction(test_video_file, model, actions, window_size=60, smoothing_window=20, threshold=0.5)'''
    

Epoch [1/16000] — Loss: 2.3070 — Acc: 10.03%
Epoch [2/16000] — Loss: 2.3007 — Acc: 10.73%
Epoch [3/16000] — Loss: 2.3014 — Acc: 11.76%
Epoch [4/16000] — Loss: 2.3056 — Acc: 8.65%
Epoch [5/16000] — Loss: 2.3080 — Acc: 10.73%
Epoch [6/16000] — Loss: 2.3017 — Acc: 10.38%
Epoch [7/16000] — Loss: 2.3099 — Acc: 6.57%
Epoch [8/16000] — Loss: 2.3002 — Acc: 11.07%
Epoch [9/16000] — Loss: 2.2990 — Acc: 10.38%
Epoch [10/16000] — Loss: 2.3131 — Acc: 11.42%
Epoch [11/16000] — Loss: 2.2956 — Acc: 10.38%
Epoch [12/16000] — Loss: 2.3030 — Acc: 10.38%
Epoch [13/16000] — Loss: 2.3035 — Acc: 11.07%
Epoch [14/16000] — Loss: 2.3002 — Acc: 8.30%
Epoch [15/16000] — Loss: 2.3090 — Acc: 10.73%
Epoch [16/16000] — Loss: 2.3171 — Acc: 12.11%
Epoch [17/16000] — Loss: 2.3027 — Acc: 8.65%
Epoch [18/16000] — Loss: 2.2970 — Acc: 10.38%
Epoch [19/16000] — Loss: 2.3055 — Acc: 9.69%
Epoch [20/16000] — Loss: 2.3073 — Acc: 9.69%
Epoch [21/16000] — Loss: 2.3045 — Acc: 8.65%
Epoch [22/16000] — Loss: 2.2951 — Acc: 7.61%
Epoch

In [10]:
# VANILLA LSTM infer_one_shot.py

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import mediapipe as mp

# =============================================================================
# 1) DEVICE
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2) FRAME SAMPLING UTILITY
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    n = len(frames)
    if n == 0:
        return []
    idxs = np.linspace(0, n - 1, target_count).round().astype(int)
    return [frames[i] for i in idxs]

def read_and_sample(video_path, target_count=60):
    cap, frames = cv2.VideoCapture(video_path), []
    while True:
        ret, f = cap.read()
        if not ret:
            break
        frames.append(f)
    cap.release()
    return sample_frames_uniformly(frames, target_count)

# =============================================================================
# 3) MEDIAPIPE KEYPOINT EXTRACTION & NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic

def extract_all_keypoints(results):
    # Pose (33×[x,y,z,vis])
    if results.pose_landmarks:
        p = np.array([[l.x, l.y, l.z, l.visibility]
                      for l in results.pose_landmarks.landmark])
        try:
            a,b = p[11,:2], p[12,:2]
            ctr = (a+b)/2; scl = np.linalg.norm(a-b) or 1.0
        except:
            ctr, scl = np.array([0.5,0.5]), 1.0
        norm_pose = np.hstack([ (p[:,:2]-ctr)/scl, p[:,2:3], p[:,3:4] ])
    else:
        norm_pose = np.zeros((33,4))
    # Face (468×[x,y,z])
    if results.face_landmarks:
        f = np.array([[l.x, l.y, l.z] for l in results.face_landmarks.landmark])
        norm_face = np.hstack([ (f[:,:2]-ctr)/scl, f[:,2:3] ])
    else:
        norm_face = np.zeros((468,3))
    # Left hand (21×[x,y,z])
    if results.left_hand_landmarks:
        lh = np.array([[l.x, l.y, l.z] for l in results.left_hand_landmarks.landmark])
        w = lh[0,:2]
        norm_lh = np.hstack([ (lh[:,:2]-w)/scl, lh[:,2:3] ])
    else:
        norm_lh = np.zeros((21,3))
    # Right hand
    if results.right_hand_landmarks:
        rh = np.array([[l.x, l.y, l.z] for l in results.right_hand_landmarks.landmark])
        w = rh[0,:2]
        norm_rh = np.hstack([ (rh[:,:2]-w)/scl, rh[:,2:3] ])
    else:
        norm_rh = np.zeros((21,3))
    return np.concatenate([
        norm_pose.flatten(),
        norm_face.flatten(),
        norm_lh.flatten(),
        norm_rh.flatten()
    ])

# =============================================================================
# 4) YOUR TRAINED LSTM MODEL CLASS
# =============================================================================
class ActionModelPyTorch(nn.Module):
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, 64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout_prob)
        self.lstm2 = nn.LSTM(64, 128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(dropout_prob)
        self.lstm3 = nn.LSTM(128, 256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.drop3 = nn.Dropout(dropout_prob)
        self.fc1   = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.drop4 = nn.Dropout(dropout_prob)
        self.fc2   = nn.Linear(64, num_classes)
    def forward(self, x):
        x,_ = self.lstm1(x); x=self.drop1(self.relu1(x))
        x,_ = self.lstm2(x); x=self.drop2(self.relu2(x))
        x,_ = self.lstm3(x); x=self.drop3(self.relu3(x))
        x    = x[:,-1,:]
        x    = self.drop4(self.relu4(self.fc1(x)))
        return self.fc2(x)

# =============================================================================
# 5) SINGLE‐SHOT EVALUATION UTILITY
# =============================================================================
def evaluate_once(video_path, model, actions, seq_len=60):
    # 1) sample frames
    frames = read_and_sample(video_path, seq_len)
    # 2) extract keypoints
    holo = mp_holistic.Holistic(min_detection_confidence=0.5,
                                min_tracking_confidence=0.5)
    kp_seq = []
    for f in frames:
        img = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
        res = holo.process(img)
        kp_seq.append(extract_all_keypoints(res))
    holo.close()
    kp_seq = np.stack(kp_seq, axis=0)                   # (60, kp_dim)
    inp    = torch.tensor(kp_seq, dtype=torch.float32) \
                    .unsqueeze(0).to(device)           # (1,60,kp_dim)
    # 3) forward
    model.eval()
    with torch.no_grad():
        logits = model(inp)
        probs  = torch.softmax(logits, dim=1)[0].cpu().numpy()
    idx = int(probs.argmax())
    print(f"Prediction: {actions[idx]}  (conf={probs[idx]:.2f})")
    print({actions[i]: float(probs[i]) for i in range(len(actions))})

# =============================================================================
# 6) MAIN
# =============================================================================
if __name__ == "__main__":
    ACTIONS = ["Death","Doctor","he","healthy","Hospital",
               "I","Medicine","Patient","she","sick"]

    # adjust this path if needed
    MODEL_PATH = "action_detection_model60_29.pth"
    VIDEO_PATH = "./videos/hospital_test.MOV"

    # instantiate & load
    # keypoint_dim = 33*4 + 468*3 + 21*3 + 21*3 = 1662
    model = ActionModelPyTorch(1662, len(ACTIONS)).to(device)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))

    # run one‐shot inference
    evaluate_once(VIDEO_PATH, model, ACTIONS)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Prediction: she  (conf=0.53)
{'Death': 0.0004955986514687538, 'Doctor': 0.0004365878994576633, 'he': 0.3322489857673645, 'healthy': 0.0021612585987895727, 'Hospital': 0.00022355471446644515, 'I': 0.10717284679412842, 'Medicine': 9.295306517742574e-05, 'Patient': 0.027279717847704887, 'she': 0.5293811559677124, 'sick': 0.0005074225482530892}


In [1]:
#ATTENTION !!! Newer model with Attention + CNN + LSTM training part

# =============================================================================
# 1. IMPORT DEPENDENCIES
# =============================================================================
import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
import random

# =============================================================================
# 2. MEDIAPIPE KEYPOINT EXTRACTION & NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic

def extract_all_keypoints(results):
    """
    Extract and normalize keypoints from MediaPipe results.
    Uses shoulder midpoint & distance for pose normalization.
    Returns flattened pose (33×4), face (468×3), left/right hand (21×3 each).
    """
    # Pose
    if results.pose_landmarks:
        pose = np.array([[lm.x, lm.y, lm.z, lm.visibility]
                         for lm in results.pose_landmarks.landmark])
        try:
            l = pose[11, :2]; r = pose[12, :2]
            center = (l + r) / 2
            scale = np.linalg.norm(l - r) or 1.0
        except:
            center = np.array([0.5,0.5]); scale = 1.0
        norm_pose = np.hstack([ (pose[:,:2] - center)/scale,
                                pose[:,2:3], pose[:,3:4] ])
    else:
        norm_pose = np.zeros((33,4))
    # Face
    if results.face_landmarks:
        face = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face[:,:2]-center)/scale, face[:,2:3] ])
    else:
        norm_face = np.zeros((468,3))
    # Left hand
    if results.left_hand_landmarks:
        lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark])
        wrist = lh[0,:2]
        norm_lh = np.hstack([ (lh[:,:2]-wrist)/scale, lh[:,2:3] ])
    else:
        norm_lh = np.zeros((21,3))
    # Right hand
    if results.right_hand_landmarks:
        rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark])
        wrist = rh[0,:2]
        norm_rh = np.hstack([ (rh[:,:2]-wrist)/scale, rh[:,2:3] ])
    else:
        norm_rh = np.zeros((21,3))
    return np.concatenate([norm_pose.flatten(),
                           norm_face.flatten(),
                           norm_lh.flatten(),
                           norm_rh.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION
# =============================================================================
def augment_sequence(seq, noise_std=0.01):
    noise = np.random.normal(0, noise_std, seq.shape)
    return seq + noise

# =============================================================================
# 4. UTILITY: SAMPLE FRAMES
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    n = len(frames)
    if n == 0: return []
    if n == target_count: return frames
    indices = [int(round(i*(n-1)/(target_count-1))) for i in range(target_count)]
    return [frames[i] for i in indices]

# =============================================================================
# 5. DATASET CLASS
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    """
    Expects: data_dir/<action>/<video_folder>/frame_##.npy
    """
    def __init__(self, data_dir, actions, seq_len=60, augment=False):
        self.data, self.labels = [], []
        self.actions, self.seq_len, self.aug = actions, seq_len, augment
        for lbl, act in enumerate(actions):
            act_dir = os.path.join(data_dir, act)
            if not os.path.isdir(act_dir): continue
            for vid in os.listdir(act_dir):
                vdir = os.path.join(act_dir, vid)
                if not os.path.isdir(vdir): continue
                files = sorted([f for f in os.listdir(vdir) if f.endswith('.npy')],
                               key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(files) != seq_len: continue
                seq = np.stack([np.load(os.path.join(vdir,f)) for f in files])
                self.data.append(seq); self.labels.append(lbl)
        self.data = np.array(self.data); self.labels = np.array(self.labels)
    def __len__(self): return len(self.labels)
    def __getitem__(self, i):
        seq = self.data[i]
        if self.aug: seq = augment_sequence(seq)
        return torch.tensor(seq, dtype=torch.float32), torch.tensor(self.labels[i])

# =============================================================================
# 6. ATTENTION MODULE
# =============================================================================
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.fc = nn.Linear(hidden_dim*2, 1)
    def forward(self, lstm_out):
        # lstm_out: (B, T, H*2)
        scores = self.fc(lstm_out)           # (B, T, 1)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * lstm_out, dim=1)  # (B, H*2)
        return context, weights

# =============================================================================
# 7. HYBRID MODEL: CNN → BiLSTM → ATTENTION → FC
# =============================================================================
class HybridModel(nn.Module):
    def __init__(self, input_dim, num_classes,
                 cnn_ch=[64,128], lstm_hidden=256):
        super().__init__()
        # Temporal CNN
        self.conv1 = nn.Conv1d(input_dim, cnn_ch[0], kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(cnn_ch[0], cnn_ch[1], kernel_size=3, padding=1)
        # BiLSTM
        self.lstm = nn.LSTM(cnn_ch[1], lstm_hidden,
                            batch_first=True, bidirectional=True)
        # Attention
        self.attn = Attention(lstm_hidden)
        # Classifier
        self.fc = nn.Linear(lstm_hidden*2, num_classes)
        self.relu = nn.ReLU()
    def forward(self, x):
        # x: (B, T=60, D)
        x = x.transpose(1,2)          # → (B, D, T)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.transpose(1,2)          # → (B, T, C)
        lstm_out, _ = self.lstm(x)    # → (B, T, H*2)
        ctx, attn_w = self.attn(lstm_out)
        logits = self.fc(ctx)         # → (B, num_classes)
        return logits, attn_w

# =============================================================================
# 8. TRAINING & EVALUATION FUNCTIONS
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(data_dir, actions, seq_len=60,
                batch_size=16, epochs=100, lr=1e-3, augment=True,
                early_stop_acc=0.92):
    """
    Train the HybridModel, but stop early if training accuracy ≥ early_stop_acc.
    """
    # 1. Prepare dataset and dataloader
    ds = SignLanguageVideoDataset(data_dir, actions, seq_len, augment=augment)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)

    # 2. Initialize model, optimizer, loss
    sample, _ = ds[0]
    model = HybridModel(sample.shape[1], len(actions)).to(device)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    crit = nn.CrossEntropyLoss()

    for ep in range(1, epochs+1):
        model.train()
        total_loss = 0

        # --- Training loop ---
        for seqs, lbls in dl:
            seqs, lbls = seqs.to(device), lbls.to(device)
            opt.zero_grad()
            out, _ = model(seqs)
            loss = crit(out, lbls)
            loss.backward()
            opt.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dl)
        print(f"Epoch {ep}/{epochs} — Loss: {avg_loss:.4f}")

        # --- Compute training accuracy for early stopping ---
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for seqs, lbls in dl:
                seqs, lbls = seqs.to(device), lbls.to(device)
                out, _ = model(seqs)
                preds = out.argmax(dim=1)
                correct += (preds == lbls).sum().item()
                total += lbls.size(0)
        train_acc = correct / total
        print(f"Epoch {ep} — Training Accuracy: {train_acc*100:.2f}%")

        # --- Early stopping check ---
        if train_acc >= early_stop_acc:
            print(f"Reached {early_stop_acc*100:.1f}% accuracy — stopping early.")
            torch.save(model.state_dict(), "attention_hybrid_model29.pth")
            return model

    # If we finish all epochs without early stop, save final model
    torch.save(model.state_dict(), "attention_hybrid_model29.pth")
    return model


def evaluate_model(model, data_dir, actions, seq_len=60, batch_size=16):
    ds = SignLanguageVideoDataset(data_dir, actions, seq_len, augment=False)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=4)
    model.eval(); correct=0; total=0
    with torch.no_grad():
        for seqs, lbls in dl:
            seqs, lbls = seqs.to(device), lbls.to(device)
            out, _ = model(seqs)
            preds = out.argmax(dim=1)
            correct += (preds==lbls).sum().item()
            total += lbls.size(0)
    acc = 100*correct/total
    print(f"Test Accuracy: {acc:.2f}%")
    # Confusion & classification report
    all_preds, all_lbls = [], []
    with torch.no_grad():
        for seqs, lbls in dl:
            seqs = seqs.to(device)
            out, _ = model(seqs)
            all_preds += out.argmax(dim=1).cpu().tolist()
            all_lbls += lbls.tolist()
    print("Confusion Matrix:\n", confusion_matrix(all_lbls, all_preds))
    print("Classification Report:\n", classification_report(all_lbls, all_preds,
          target_names=actions))
    return acc

# =============================================================================
# 9. REAL‑TIME SLIDING WINDOW PREDICTION
# =============================================================================
def prob_viz(probs, actions, frame, colors):
    out = frame.copy()
    for i,p in enumerate(probs):
        w = int(p*200)
        cv2.rectangle(out, (0,60+40*i), (w,90+40*i), colors[i%len(colors)], -1)
        cv2.putText(out, f"{actions[i]}: {p:.2f}", (10,85+40*i),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    return out

def sliding_window_prediction(video_path, model, actions,
                              window_size=60, smooth_w=10, thresh=0.5):
    colors = [(245,117,16),(117,245,16),(16,117,245),(0,255,0)]
    seq=[]; preds=[]; sentence=[]
    cap = cv2.VideoCapture(video_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holo:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES,0); continue
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable=False
            res = holo.process(img)
            img.flags.writeable=True
            frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            kp = extract_all_keypoints(res)
            seq.append(kp)
            if len(seq)>window_size: seq.pop(0)
            if len(seq)==window_size:
                arr = np.stack(seq)
                tensor = torch.tensor(arr, dtype=torch.float32).unsqueeze(0).to(device)
                model.eval()
                with torch.no_grad():
                    out, _ = model(tensor)
                    prob = torch.softmax(out,1).cpu().numpy()[0]
                    idx = prob.argmax()
                    preds.append(idx)
                if len(preds)>=smooth_w:
                    recent = preds[-smooth_w:]
                    if len(set(recent))==1 and prob[idx]>thresh:
                        if not sentence or sentence[-1]!=actions[idx]:
                            sentence.append(actions[idx])
                vis = prob_viz(prob, actions, frame, colors)
                cv2.rectangle(vis, (0,0),(640,40),(245,117,16),-1)
                cv2.putText(vis, ' '.join(sentence),(3,30),
                            cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
                cv2.imshow("SLR", vis)
            else:
                cv2.imshow("SLR", frame)
            if cv2.waitKey(10)&0xFF==ord('q'): break
    cap.release(); cv2.destroyAllWindows()

# =============================================================================
# 10. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    actions = ["Death","Doctor","he","healthy","Hospital",
               "I","Medicine","Patient","she","sick"]
    extracted_keypoints_dir = "./extracted_keypoints"

    # Train
    model = train_model(extracted_keypoints_dir, actions,
                        seq_len=60, batch_size=16,
                        epochs=10000, lr=1e-3, augment=True)

    # Evaluate
    evaluate_model(model, extracted_keypoints_dir, actions,
                   seq_len=60, batch_size=16)

    # Real‑time prediction demo (uncomment and set path)
    # sliding_window_prediction("./test_video.mp4", model, actions)


Epoch 1/10000 — Loss: 2.3267
Epoch 1 — Training Accuracy: 10.03%
Epoch 2/10000 — Loss: 2.3116
Epoch 2 — Training Accuracy: 10.03%
Epoch 3/10000 — Loss: 2.3075
Epoch 3 — Training Accuracy: 10.03%
Epoch 4/10000 — Loss: 2.3104
Epoch 4 — Training Accuracy: 11.07%
Epoch 5/10000 — Loss: 2.3068
Epoch 5 — Training Accuracy: 15.22%
Epoch 6/10000 — Loss: 2.3024
Epoch 6 — Training Accuracy: 11.76%
Epoch 7/10000 — Loss: 2.2579
Epoch 7 — Training Accuracy: 11.76%
Epoch 8/10000 — Loss: 2.2798
Epoch 8 — Training Accuracy: 14.19%
Epoch 9/10000 — Loss: 2.2193
Epoch 9 — Training Accuracy: 18.34%
Epoch 10/10000 — Loss: 2.1421
Epoch 10 — Training Accuracy: 23.53%
Epoch 11/10000 — Loss: 1.9596
Epoch 11 — Training Accuracy: 21.11%
Epoch 12/10000 — Loss: 1.8952
Epoch 12 — Training Accuracy: 31.14%
Epoch 13/10000 — Loss: 1.7861
Epoch 13 — Training Accuracy: 37.02%
Epoch 14/10000 — Loss: 1.8320
Epoch 14 — Training Accuracy: 37.37%
Epoch 15/10000 — Loss: 1.6577
Epoch 15 — Training Accuracy: 43.94%
Epoch 16/1000

In [3]:
sliding_window_prediction("./videos/SourikSampleVideo.mp4", model, actions)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

In [9]:
#sliding window with correct import statements

# =============================================================================
# 1. IMPORT DEPENDENCIES
# =============================================================================
import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
import random

# =============================================================================
# 2. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 3. MEDIAPIPE KEYPOINT EXTRACTION & NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic

def extract_all_keypoints(results):
    """
    Extract and normalize keypoints from MediaPipe results.
    """
    # Pose
    if results.pose_landmarks:
        pose = np.array([[lm.x, lm.y, lm.z, lm.visibility]
                         for lm in results.pose_landmarks.landmark])
        try:
            l, r = pose[11, :2], pose[12, :2]
            center = (l + r) / 2
            scale = np.linalg.norm(l - r) or 1.0
        except:
            center, scale = np.array([0.5,0.5]), 1.0
        norm_pose = np.hstack([
            (pose[:, :2] - center) / scale,
            pose[:, 2:3],
            pose[:, 3:4]
        ])
    else:
        norm_pose = np.zeros((33,4))
    # Face
    if results.face_landmarks:
        face = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark])
        norm_face = np.hstack([
            (face[:, :2] - center) / scale,
            face[:, 2:3]
        ])
    else:
        norm_face = np.zeros((468,3))
    # Left hand
    if results.left_hand_landmarks:
        lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark])
        wrist = lh[0,:2]
        norm_lh = np.hstack([(lh[:,:2] - wrist) / scale, lh[:,2:3]])
    else:
        norm_lh = np.zeros((21,3))
    # Right hand
    if results.right_hand_landmarks:
        rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark])
        wrist = rh[0,:2]
        norm_rh = np.hstack([(rh[:,:2] - wrist) / scale, rh[:,2:3]])
    else:
        norm_rh = np.zeros((21,3))
    return np.concatenate([
        norm_pose.flatten(),
        norm_face.flatten(),
        norm_lh.flatten(),
        norm_rh.flatten()
    ])

# =============================================================================
# 4. DATA AUGMENTATION
# =============================================================================
def augment_sequence(seq, noise_std=0.01):
    noise = np.random.normal(0, noise_std, seq.shape)
    return seq + noise

# =============================================================================
# 5. FRAME SAMPLER
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    n = len(frames)
    if n == 0: return []
    if n == target_count: return frames
    idxs = [int(round(i*(n-1)/(target_count-1))) for i in range(target_count)]
    return [frames[i] for i in idxs]

# =============================================================================
# 6. DATASET CLASS
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    """
    Expects: data_dir/<action>/<video_folder>/frame_##.npy
    """
    def __init__(self, data_dir, actions, seq_len=60, augment=False):
        self.data, self.labels = [], []
        self.actions, self.seq_len, self.aug = actions, seq_len, augment
        for lbl, act in enumerate(actions):
            act_dir = os.path.join(data_dir, act)
            if not os.path.isdir(act_dir): 
                continue
            for vid in os.listdir(act_dir):
                vdir = os.path.join(act_dir, vid)
                if not os.path.isdir(vdir):
                    continue
                files = sorted(
                    [f for f in os.listdir(vdir) if f.endswith('.npy')],
                    key=lambda x: int(x.split('_')[1].split('.')[0])
                )
                if len(files) != seq_len: 
                    continue
                seq = np.stack([np.load(os.path.join(vdir, f)) for f in files])
                self.data.append(seq)
                self.labels.append(lbl)
        self.data   = np.array(self.data)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        seq = self.data[i]
        if self.aug:
            seq = augment_sequence(seq)
        return (
            torch.tensor(seq, dtype=torch.float32),
            torch.tensor(self.labels[i], dtype=torch.long)
        )

# =============================================================================
# 7. ATTENTION MODULE
# =============================================================================
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, lstm_out):
        scores = self.fc(lstm_out)            # (B, T, 1)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * lstm_out, dim=1)  # (B, H*2)
        return context, weights

# =============================================================================
# 8. LSTM-ONLY MODEL (for loader option)
# =============================================================================
class ActionModelPyTorch(nn.Module):
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, 64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        self.lstm2 = nn.LSTM(64, 128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.lstm3 = nn.LSTM(128, 256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        out,_ = self.lstm1(x)
        out = self.relu1(out); out = self.dropout1(out)
        out,_ = self.lstm2(out)
        out = self.relu2(out); out = self.dropout2(out)
        out,_ = self.lstm3(out)
        out = self.relu3(out); out = self.dropout3(out)
        out = out[:, -1, :]
        out = self.fc1(out); out = self.relu4(out); out = self.dropout4(out)
        return self.fc2(out), None

# =============================================================================
# 9. HYBRID MODEL (CNN→BiLSTM→ATTN→FC)
# =============================================================================
class HybridModel(nn.Module):
    def __init__(self, input_dim, num_classes,
                 cnn_ch=[64,128], lstm_hidden=256):
        super().__init__()
        # Temporal CNN
        self.conv1 = nn.Conv1d(input_dim, cnn_ch[0], kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(cnn_ch[0], cnn_ch[1], kernel_size=3, padding=1)
        # BiLSTM
        self.lstm = nn.LSTM(cnn_ch[1], lstm_hidden,
                            batch_first=True, bidirectional=True)
        # Attention
        self.attn = Attention(lstm_hidden)
        # Classifier
        self.fc = nn.Linear(lstm_hidden * 2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.transpose(1, 2)              # (B, D, T)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.transpose(1, 2)              # (B, T, C)
        lstm_out, _ = self.lstm(x)         # (B, T, H*2)
        ctx, attn_w = self.attn(lstm_out)
        return self.fc(ctx), attn_w

# =============================================================================
# 10. MODEL LOADER HELPER
# =============================================================================
def load_model(model_name: str,
               model_path: str,
               input_dim: int,
               num_classes: int) -> nn.Module:
    """
    Instantiate and load a model by name.
      model_name: "hybrid" or "lstm"
      model_path: path to .pth file
      input_dim: feature dimension per frame
      num_classes: number of output classes
    """
    if model_name.lower() == "hybrid":
        model = HybridModel(input_dim, num_classes)
    elif model_name.lower() == "lstm":
        model = ActionModelPyTorch(input_dim, num_classes)
    else:
        raise ValueError(f"Unknown model_name '{model_name}'. Choose 'hybrid' or 'lstm'.")
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device).eval()
    return model

# =============================================================================
# 11. SLIDING WINDOW PREDICTION (REAL‑TIME)
# =============================================================================
def prob_viz(probs, actions, frame, colors):
    out = frame.copy()
    for i, p in enumerate(probs):
        w = int(p * 200)
        cv2.rectangle(out, (0, 60 + 40*i), (w, 90 + 40*i), colors[i % len(colors)], -1)
        cv2.putText(out, f"{actions[i]}: {p:.2f}", (10, 85 + 40*i),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    return out

def sliding_window_prediction(video_path: str,
                              model: nn.Module,
                              actions: list[str],
                              window_size=60,
                              smooth_w=10,
                              thresh=0.5):
    colors = [(245,117,16),(117,245,16),(16,117,245),(0,255,0)]
    seq, preds, sentence = [], [], []
    cap = cv2.VideoCapture(video_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holo:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False
            res = holo.process(img)
            img.flags.writeable = True
            vis_frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            kp = extract_all_keypoints(res)
            seq.append(kp)
            if len(seq) > window_size:
                seq.pop(0)
            if len(seq) == window_size:
                arr = np.stack(seq)
                tensor = torch.tensor(arr, dtype=torch.float32).unsqueeze(0).to(device)
                with torch.no_grad():
                    out, _ = model(tensor)
                    prob = torch.softmax(out, dim=1).cpu().numpy()[0]
                    idx = prob.argmax()
                    preds.append(idx)
                if len(preds) >= smooth_w:
                    recent = preds[-smooth_w:]
                    if len(set(recent)) == 1 and prob[idx] > thresh:
                        if not sentence or sentence[-1] != actions[idx]:
                            sentence.append(actions[idx])
                vis = prob_viz(prob, actions, vis_frame, colors)
                cv2.rectangle(vis, (0,0), (640,40), (245,117,16), -1)
                cv2.putText(vis, ' '.join(sentence), (3,30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                cv2.imshow("SLR", vis)
            else:
                cv2.imshow("SLR", vis_frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# =============================================================================
# 12. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    actions = ["Death","Doctor","he","healthy","Hospital",
               "I","Medicine","Patient","she","sick"]
    extracted_keypoints_dir = "./extracted_keypoints"

    # -------------------------------------------------------------------------
    # TRAINING (unchanged from before, or you can call your train_model here)
    # -------------------------------------------------------------------------
    # from earlier cell / script:
    # model = train_model(extracted_keypoints_dir, actions, seq_len=60, batch_size=16, epochs=250, lr=1e-3, augment=True)

    # -------------------------------------------------------------------------
    # LOADING & REAL‑TIME DEMO
    # -------------------------------------------------------------------------
    # Determine input_dim and num_classes from your dataset
    sample_seq, _ = SignLanguageVideoDataset(extracted_keypoints_dir, actions, seq_len=60).__getitem__(0)
    input_dim  = sample_seq.shape[1]
    num_classes = len(actions)

    # Load the hybrid model
    model_name = "hybrid"                                # or "lstm"
    model_path = "attention_hybrid_model29.pth"                      # path to your .pth file
    model = load_model(model_name, model_path, input_dim, num_classes)

    # Real‑time sliding-window prediction
    sliding_window_prediction("./videos/SourikSampleVideo.mp4", model, actions)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

In [5]:
# True ResNet → FNet → LSTM Continuous ISL Recognition Pipeline (fixed)
# =============================================================================

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp_holistic
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models.feature_extraction import create_feature_extractor
from sklearn.metrics import confusion_matrix, classification_report

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. FRAME SAMPLING
# =============================================================================
def sample_frames(video_path, target_count=60):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    n = len(frames)
    if n == 0:
        return []
    if n == target_count:
        return frames
    idxs = [int(round(i*(n-1)/(target_count-1))) for i in range(target_count)]
    return [frames[i] for i in idxs]

# =============================================================================
# 3. DATASET: RAW VIDEOS → RESNET FEATURES
# =============================================================================
class VideoResNetDataset(Dataset):
    """
    Loads videos from data_dir/<action>/*.mp4, samples 60 frames,
    extracts 512‑d ResNet features per frame, returns (60×512) tensor.
    """
    def __init__(self, data_dir, actions, seq_len=60, augment=False):
        self.samples = []
        self.actions = actions
        self.seq_len = seq_len
        self.augment = augment

        # ImageNet preprocessing
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((224,224)),
            T.ToTensor(),
            T.Normalize([0.485,0.456,0.406],
                        [0.229,0.224,0.225])
        ])

        # Frozen ResNet‑18 up to avgpool → 512‑dim
        weights = ResNet18_Weights.IMAGENET1K_V1
        resnet = resnet18(weights=weights).to(device).eval()
        self.fe = create_feature_extractor(resnet, return_nodes=['avgpool'])

        # Gather (video_path, label)
        for lbl, act in enumerate(actions):
            act_dir = os.path.join(data_dir, act)
            if not os.path.isdir(act_dir):
                continue
            for fn in os.listdir(act_dir):
                if fn.lower().endswith((".mp4", ".avi", ".mov")):
                    self.samples.append((os.path.join(act_dir, fn), lbl))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        frames = sample_frames(video_path, self.seq_len)
        feats = []
        with torch.no_grad():
            for img in frames:
                t = self.transform(img).unsqueeze(0).to(device)
                out = self.fe(t)['avgpool']        # (1,512,1,1)
                feats.append(out.view(-1).cpu().numpy())
        seq = np.stack(feats)                   # (60,512)
        if self.augment:
            seq += np.random.normal(0, 0.01, seq.shape)
        return torch.tensor(seq, dtype=torch.float32), label

# =============================================================================
# 4. FNet TOKEN‑MIXING BLOCK
# =============================================================================
class FNetBlock(nn.Module):
    """
    Token mixing via real FFT along the time dimension.
    Input: (B, 60, 512) → Output: (B, 60, 512)
    """
    def forward(self, x):
        x_f = torch.fft.rfft(x, dim=1)
        x_if = torch.fft.irfft(x_f, n=x.size(1), dim=1)
        return x_if

# =============================================================================
# 5. MODEL: RESNET → FNET → BiLSTM → FC
# =============================================================================
class ResNetFNetLSTM(nn.Module):
    def __init__(self, feat_dim, num_classes, lstm_hidden=256, bidir=True):
        super().__init__()
        self.fnet = FNetBlock()
        self.lstm = nn.LSTM(input_size=feat_dim,
                            hidden_size=lstm_hidden,
                            batch_first=True,
                            bidirectional=bidir)
        self.fc = nn.Linear(lstm_hidden*(2 if bidir else 1),
                            num_classes)

    def forward(self, x):
        # x: (B, 60, 512)
        x = self.fnet(x)               # (B, 60, 512)
        out, _ = self.lstm(x)          # (B, 60, H*dirs)
        last = out[:, -1, :]           # (B, H*dirs)
        return self.fc(last)           # (B, num_classes)

# =============================================================================
# 6. TRAINING WITH EARLY STOPPING
# =============================================================================
def train_model(data_dir, actions, seq_len=60,
                batch_size=8, epochs=300, lr=1e-4, augment=True):
    dataset = VideoResNetDataset(data_dir, actions, seq_len, augment)
    loader  = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0  # <— set to 0 to avoid multiprocessing pickle issues
    )

    sample, _ = dataset[0]
    model = ResNetFNetLSTM(sample.shape[1], len(actions)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        print("Staring training model...")
        total_loss = correct = total = 0
        for seqs, labels in loader:
            seqs, labels = seqs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(seqs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        acc = 100 * correct / total
        print(f"Epoch {epoch}/{epochs} — "
              f"Loss: {total_loss/len(loader):.4f} — Acc: {acc:.2f}%")

        if epoch >= 10 and acc >= 90.0:
            print(f"Early stopping at epoch {epoch} (train acc {acc:.2f}%)")
            break

    torch.save(model.state_dict(), "resnet_fnet_lstm29.pth")
    print("Saved model to resnet_fnet_lstm.pth")
    return model

# =============================================================================
# 7. EVALUATION
# =============================================================================
def evaluate_model(model, data_dir, actions, seq_len=60, batch_size=8):
    dataset = VideoResNetDataset(data_dir, actions, seq_len, augment=False)
    loader  = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0  # match training
    )
    print("Starting model evaluation...")
    model.eval()
    correct = total = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for seqs, labels in loader:
            seqs, labels = seqs.to(device), labels.to(device)
            outputs = model(seqs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds += preds.cpu().tolist()
            all_labels += labels.cpu().tolist()

    acc = 100 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    print("Classification Report:\n",
          classification_report(all_labels, all_preds,
                                target_names=actions))

# =============================================================================
# 8. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    actions = [
        "Death","Doctor","he","healthy","Hospital",
        "I","Medicine","Patient","she","sick"
    ]
    data_dir = "./dataset"  # each action folder contains raw videos

    # Train (with early stopping)
    model = train_model(
        data_dir, actions,
        seq_len=60, batch_size=8,
        epochs=500, lr=1e-4, augment=True
    )

    # Evaluate on held‑out set
    evaluate_model(
        model, data_dir, actions,
        seq_len=60, batch_size=8
    )


Staring training model...
Epoch 1/50 — Loss: 2.2804 — Acc: 16.57%
Staring training model...
Epoch 2/50 — Loss: 2.0831 — Acc: 34.25%
Staring training model...
Epoch 3/50 — Loss: 1.8983 — Acc: 36.46%
Staring training model...
Epoch 4/50 — Loss: 1.6747 — Acc: 41.44%
Staring training model...
Epoch 5/50 — Loss: 1.4523 — Acc: 52.49%
Staring training model...
Epoch 6/50 — Loss: 1.2728 — Acc: 51.93%
Staring training model...
Epoch 7/50 — Loss: 1.1410 — Acc: 54.14%
Staring training model...
Epoch 8/50 — Loss: 1.0383 — Acc: 56.35%
Staring training model...
Epoch 9/50 — Loss: 0.9827 — Acc: 56.35%
Staring training model...
Epoch 10/50 — Loss: 0.9250 — Acc: 59.12%
Staring training model...
Epoch 11/50 — Loss: 0.8939 — Acc: 59.67%
Staring training model...
Epoch 12/50 — Loss: 0.8579 — Acc: 60.77%
Staring training model...
Epoch 13/50 — Loss: 0.8145 — Acc: 70.72%
Staring training model...
Epoch 14/50 — Loss: 0.8010 — Acc: 68.51%
Staring training model...
Epoch 15/50 — Loss: 0.8205 — Acc: 61.33%
Star

In [1]:
# Fused ResNet + Pre-extracted MediaPipe Keypoints → FNet → BiLSTM ISL Recognizer
# =============================================================================

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models.feature_extraction import create_feature_extractor
from sklearn.metrics import confusion_matrix, classification_report

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. FRAME SAMPLING UTIL
# =============================================================================
def sample_frames(video_path, target_count=60):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    n = len(frames)
    if n == 0:
        return []
    if n == target_count:
        return frames
    idxs = [int(round(i*(n-1)/(target_count-1))) for i in range(target_count)]
    return [frames[i] for i in idxs]

# =============================================================================
# 3. DATASET: Raw Video + Extracted Keypoints
# =============================================================================
class FusedVideoKeypointDataset(Dataset):
    """
    Expects:
      raw_videos_dir/<action>/<video_name>.mp4
      keypoints_dir/<action>/<video_name>/frame_0.npy ... frame_59.npy
    Returns per-sample:
      R: (60, 512) ResNet features
      K: (60, keypoint_dim) loaded landmarks
      label: int
    """
    def __init__(self, raw_videos_dir, keypoints_dir, actions,
                 seq_len=60, augment=False):
        self.samples = []  # list of tuples (video_path, keypt_folder, label)
        self.actions  = actions
        self.seq_len   = seq_len
        self.augment   = augment

        # ImageNet preprocessing
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((224,224)),
            T.ToTensor(),
            T.Normalize([0.485,0.456,0.406],
                        [0.229,0.224,0.225])
        ])

        # Frozen ResNet-18 up to 'avgpool' → 512-d
        weights = ResNet18_Weights.IMAGENET1K_V1
        resnet = resnet18(weights=weights).to(device).eval()
        self.fe = create_feature_extractor(resnet, return_nodes=['avgpool'])

        # Build sample list
        for lbl, act in enumerate(actions):
            rv_act = os.path.join(raw_videos_dir, act)
            kp_act = os.path.join(keypoints_dir, act)
            if not os.path.isdir(rv_act) or not os.path.isdir(kp_act):
                continue
            for vp in os.listdir(rv_act):
                name, ext = os.path.splitext(vp)
                if ext.lower() not in ('.mp4','.avi','.mov'):
                    continue
                video_path = os.path.join(rv_act, vp)
                kp_folder  = os.path.join(kp_act, name)
                # ensure keypoint folder exists and has seq_len .npy files
                if not os.path.isdir(kp_folder):
                    continue
                files = [f for f in os.listdir(kp_folder) if f.endswith('.npy')]
                if len(files) != seq_len:
                    continue
                self.samples.append((video_path, kp_folder, lbl))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, kp_folder, label = self.samples[idx]

        # 1. Sample frames & extract ResNet features
        frames = sample_frames(video_path, self.seq_len)
        R = []
        with torch.no_grad():
            for img in frames:
                t = self.transform(img).unsqueeze(0).to(device)
                out = self.fe(t)['avgpool']       # (1,512,1,1)
                R.append(out.view(-1).cpu().numpy())
        R = np.stack(R)  # (60,512)

        # 2. Load keypoints from .npy files
        kp_files = sorted(os.listdir(kp_folder),
                          key=lambda x: int(x.split('_')[1].split('.')[0]))
        K = [np.load(os.path.join(kp_folder, f)) for f in kp_files]  # list of (keypoint_dim,)
        K = np.stack(K)  # (60, keypoint_dim)

        # 3. Optional augmentation
        if self.augment:
            R += np.random.normal(0,0.01,R.shape)
            K += np.random.normal(0,0.01,K.shape)

        # 4. Return tensors
        return (torch.tensor(R, dtype=torch.float32),
                torch.tensor(K, dtype=torch.float32),
                label)

# =============================================================================
# 4. FNet TOKEN-MIXING BLOCK
# =============================================================================
class FNetBlock(nn.Module):
    def forward(self, x):
        x_f = torch.fft.rfft(x, dim=1)
        x_if = torch.fft.irfft(x_f, n=x.size(1), dim=1)
        return x_if

# =============================================================================
# 5. FUSED MODEL: Project → Fuse → FNet → BiLSTM → FC
# =============================================================================
class FusedModel(nn.Module):
    def __init__(self, keypoint_dim, num_classes, lstm_hidden=256):
        super().__init__()
        # Project each modality to 256
        self.resnet_proj   = nn.Linear(512,   256)
        self.landmark_proj = nn.Linear(keypoint_dim, 256)
        self.relu          = nn.ReLU()

        self.fnet = FNetBlock()
        self.lstm = nn.LSTM(input_size=512,
                            hidden_size=lstm_hidden,
                            batch_first=True,
                            bidirectional=True)

        self.classifier = nn.Linear(lstm_hidden*2, num_classes)

    def forward(self, R, K):
        # R: (B,60,512), K: (B,60,keypoint_dim)
        Rp = self.relu(self.resnet_proj(R))     # (B,60,256)
        Kp = self.relu(self.landmark_proj(K))   # (B,60,256)
        x  = torch.cat([Rp, Kp], dim=-1)        # (B,60,512)

        x, _ = self.lstm(self.fnet(x))          # (B,60,512)
        h = x[:, -1, :]                         # (B,512)
        return self.classifier(h)               # (B,num_classes)

# =============================================================================
# 6. TRAIN & EARLY STOPPING
# =============================================================================
def train_model(raw_videos_dir, kp_dir, actions,
                seq_len=60, batch_size=8,
                epochs=300, lr=1e-4, augment=True):
    ds = FusedVideoKeypointDataset(raw_videos_dir, kp_dir, actions,
                                   seq_len, augment)
    dl = DataLoader(ds, batch_size=batch_size,
                    shuffle=True, num_workers=0)

    # instantiate model
    sample_R, sample_K, _ = ds[0]
    model = FusedModel(sample_K.shape[1], len(actions)).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    crit  = nn.CrossEntropyLoss()

    for ep in range(1, epochs+1):
        model.train()
        tot_loss = corr = tot = 0
        for R,K,labels in dl:
            R, K, labels = R.to(device), K.to(device), labels.to(device)
            opt.zero_grad()
            out = model(R,K)
            loss = crit(out, labels)
            loss.backward()
            opt.step()

            tot_loss += loss.item()
            preds = out.argmax(dim=1)
            corr   += (preds == labels).sum().item()
            tot    += labels.size(0)

        acc = 100 * corr / tot
        print(f"Epoch {ep}/{epochs} — Loss: {tot_loss/len(dl):.4f} — Acc: {acc:.2f}%")
        if ep>=20 and acc>=95.0:
            print(f"Early stopping at epoch {ep} (acc {acc:.2f}%)")
            break

    torch.save(model.state_dict(), "fused_model29.pth")
    print("Saved fused_model29.pth")
    return model

# =============================================================================
# 7. EVALUATION
# =============================================================================
def evaluate_model(model, raw_videos_dir, kp_dir, actions,
                   seq_len=60, batch_size=8):
    ds = FusedVideoKeypointDataset(raw_videos_dir, kp_dir, actions,
                                   seq_len, augment=False)
    dl = DataLoader(ds, batch_size=batch_size,
                    shuffle=False, num_workers=0)
    model.eval()
    corr = tot = 0
    all_p, all_l = [], []
    with torch.no_grad():
        for R,K,labels in dl:
            R, K, labels = R.to(device), K.to(device), labels.to(device)
            out = model(R,K)
            preds = out.argmax(dim=1)
            corr += (preds == labels).sum().item()
            tot  += labels.size(0)
            all_p += preds.cpu().tolist()
            all_l += labels.cpu().tolist()

    acc = 100 * corr / tot
    print(f"Test Acc: {acc:.2f}%")
    print("CM:\n", confusion_matrix(all_l, all_p))
    print("Report:\n", classification_report(all_l, all_p,
                                             target_names=actions))

# =============================================================================
# 8. MAIN
# =============================================================================
if __name__=="__main__":
    actions       = ["Death","Doctor","he","healthy","Hospital",
                     "I","Medicine","Patient","she","sick"]
    raw_videos_dir = "./dataset"
    kp_dir         = "./extracted_keypoints"

    model = train_model(raw_videos_dir, kp_dir, actions,
                        seq_len=60, batch_size=8,
                        epochs=300, lr=1e-4, augment=True)

    evaluate_model(model, raw_videos_dir, kp_dir, actions,
                   seq_len=60, batch_size=8)


Epoch 1/300 — Loss: 2.2739 — Acc: 21.80%
Epoch 2/300 — Loss: 2.1175 — Acc: 43.94%
Epoch 3/300 — Loss: 1.7768 — Acc: 49.13%
Epoch 4/300 — Loss: 1.4927 — Acc: 50.87%
Epoch 5/300 — Loss: 1.2785 — Acc: 55.36%
Epoch 6/300 — Loss: 1.1569 — Acc: 56.75%
Epoch 7/300 — Loss: 1.0260 — Acc: 68.86%
Epoch 8/300 — Loss: 1.0648 — Acc: 61.94%
Epoch 9/300 — Loss: 0.8776 — Acc: 68.17%
Epoch 10/300 — Loss: 0.9027 — Acc: 64.71%
Epoch 11/300 — Loss: 0.7997 — Acc: 67.82%
Epoch 12/300 — Loss: 0.8453 — Acc: 69.55%
Epoch 13/300 — Loss: 0.7285 — Acc: 70.59%
Epoch 14/300 — Loss: 0.6243 — Acc: 73.36%
Epoch 15/300 — Loss: 0.5860 — Acc: 74.05%
Epoch 16/300 — Loss: 0.5446 — Acc: 74.05%
Epoch 17/300 — Loss: 0.5144 — Acc: 77.16%
Epoch 18/300 — Loss: 0.5126 — Acc: 74.39%
Epoch 19/300 — Loss: 0.5530 — Acc: 76.82%
Epoch 20/300 — Loss: 0.5226 — Acc: 75.09%
Epoch 21/300 — Loss: 0.5105 — Acc: 74.05%
Epoch 22/300 — Loss: 0.5410 — Acc: 74.39%
Epoch 23/300 — Loss: 0.5342 — Acc: 74.74%
Epoch 24/300 — Loss: 0.4986 — Acc: 76.47%
E

In [8]:
#Evaluate with entire dataset test for fused_model.pth

# =============================================================================
# Fused ResNet + Pre-extracted MediaPipe Keypoints → FNet → BiLSTM ISL Recognizer
# =============================================================================

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models.feature_extraction import create_feature_extractor
from sklearn.metrics import confusion_matrix, classification_report

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. FRAME SAMPLING UTIL
# =============================================================================
def sample_frames(video_path, target_count=60):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    n = len(frames)
    if n == 0:
        return []
    if n == target_count:
        return frames
    idxs = [int(round(i*(n-1)/(target_count-1))) for i in range(target_count)]
    return [frames[i] for i in idxs]

# =============================================================================
# 3. DATASET: Raw Video + Extracted Keypoints
# =============================================================================
class FusedVideoKeypointDataset(Dataset):
    """
    Expects:
      raw_videos_dir/<action>/<video_name>.mp4
      keypoints_dir/<action>/<video_name>/frame_0.npy ... frame_59.npy
    Returns per-sample:
      R: (60, 512) ResNet features
      K: (60, keypoint_dim) loaded landmarks
      label: int
    """
    def __init__(self, raw_videos_dir, keypoints_dir, actions,
                 seq_len=60, augment=False):
        self.samples = []  # list of tuples (video_path, keypt_folder, label)
        self.actions  = actions
        self.seq_len   = seq_len
        self.augment   = augment

        # ImageNet preprocessing
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((224,224)),
            T.ToTensor(),
            T.Normalize([0.485,0.456,0.406],
                        [0.229,0.224,0.225])
        ])

        # Frozen ResNet-18 up to 'avgpool' → 512-d
        weights = ResNet18_Weights.IMAGENET1K_V1
        resnet = resnet18(weights=weights).to(device).eval()
        self.fe = create_feature_extractor(resnet, return_nodes=['avgpool'])

        # Build sample list
        for lbl, act in enumerate(actions):
            rv_act = os.path.join(raw_videos_dir, act)
            kp_act = os.path.join(keypoints_dir, act)
            if not os.path.isdir(rv_act) or not os.path.isdir(kp_act):
                continue
            for vp in os.listdir(rv_act):
                name, ext = os.path.splitext(vp)
                if ext.lower() not in ('.mp4','.avi','.mov'):
                    continue
                video_path = os.path.join(rv_act, vp)
                kp_folder  = os.path.join(kp_act, name)
                # ensure keypoint folder exists and has seq_len .npy files
                if not os.path.isdir(kp_folder):
                    continue
                files = [f for f in os.listdir(kp_folder) if f.endswith('.npy')]
                if len(files) != seq_len:
                    continue
                self.samples.append((video_path, kp_folder, lbl))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, kp_folder, label = self.samples[idx]

        # 1. Sample frames & extract ResNet features
        frames = sample_frames(video_path, self.seq_len)
        R = []
        with torch.no_grad():
            for img in frames:
                t = self.transform(img).unsqueeze(0).to(device)
                out = self.fe(t)['avgpool']       # (1,512,1,1)
                R.append(out.view(-1).cpu().numpy())
        R = np.stack(R)  # (60,512)

        # 2. Load keypoints from .npy files
        kp_files = sorted(os.listdir(kp_folder),
                          key=lambda x: int(x.split('_')[1].split('.')[0]))
        K = [np.load(os.path.join(kp_folder, f)) for f in kp_files]  # list of (keypoint_dim,)
        K = np.stack(K)  # (60, keypoint_dim)

        # 3. Optional augmentation
        if self.augment:
            R += np.random.normal(0,0.01,R.shape)
            K += np.random.normal(0,0.01,K.shape)

        # 4. Return tensors
        return (torch.tensor(R, dtype=torch.float32),
                torch.tensor(K, dtype=torch.float32),
                label)

# =============================================================================
# 4. FNet TOKEN-MIXING BLOCK
# =============================================================================
class FNetBlock(nn.Module):
    def forward(self, x):
        x_f = torch.fft.rfft(x, dim=1)
        x_if = torch.fft.irfft(x_f, n=x.size(1), dim=1)
        return x_if

# =============================================================================
# 5. FUSED MODEL: Project → Fuse → FNet → BiLSTM → FC
# =============================================================================
class FusedModel(nn.Module):
    def __init__(self, keypoint_dim, num_classes, lstm_hidden=256):
        super().__init__()
        # Project each modality to 256
        self.resnet_proj   = nn.Linear(512,   256)
        self.landmark_proj = nn.Linear(keypoint_dim, 256)
        self.relu          = nn.ReLU()

        self.fnet = FNetBlock()
        self.lstm = nn.LSTM(input_size=512,
                            hidden_size=lstm_hidden,
                            batch_first=True,
                            bidirectional=True)

        self.classifier = nn.Linear(lstm_hidden*2, num_classes)

    def forward(self, R, K):
        # R: (B,60,512), K: (B,60,keypoint_dim)
        Rp = self.relu(self.resnet_proj(R))     # (B,60,256)
        Kp = self.relu(self.landmark_proj(K))   # (B,60,256)
        x  = torch.cat([Rp, Kp], dim=-1)        # (B,60,512)

        x, _ = self.lstm(self.fnet(x))          # (B,60,512)
        h = x[:, -1, :]                         # (B,512)
        return self.classifier(h)               # (B,num_classes)

# =============================================================================
# 6. TRAIN & EARLY STOPPING
# =============================================================================
def train_model(raw_videos_dir, kp_dir, actions,
                seq_len=60, batch_size=8,
                epochs=300, lr=1e-4, augment=True):
    ds = FusedVideoKeypointDataset(raw_videos_dir, kp_dir, actions,
                                   seq_len, augment)
    dl = DataLoader(ds, batch_size=batch_size,
                    shuffle=True, num_workers=0)

    # instantiate model
    sample_R, sample_K, _ = ds[0]
    model = FusedModel(sample_K.shape[1], len(actions)).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    crit  = nn.CrossEntropyLoss()

    for ep in range(1, epochs+1):
        model.train()
        tot_loss = corr = tot = 0
        for R,K,labels in dl:
            R, K, labels = R.to(device), K.to(device), labels.to(device)
            opt.zero_grad()
            out = model(R,K)
            loss = crit(out, labels)
            loss.backward()
            opt.step()

            tot_loss += loss.item()
            preds = out.argmax(dim=1)
            corr   += (preds == labels).sum().item()
            tot    += labels.size(0)

        acc = 100 * corr / tot
        print(f"Epoch {ep}/{epochs} — Loss: {tot_loss/len(dl):.4f} — Acc: {acc:.2f}%")
        if ep>=20 and acc>=90.0:
            print(f"Early stopping at epoch {ep} (acc {acc:.2f}%)")
            break

    torch.save(model.state_dict(), "fused_model.pth")
    print("Saved fused_model.pth")
    return model

# =============================================================================
# 7. EVALUATION
# =============================================================================
def evaluate_model(model, raw_videos_dir, kp_dir, actions,
                   seq_len=60, batch_size=8):
    ds = FusedVideoKeypointDataset(raw_videos_dir, kp_dir, actions,
                                   seq_len, augment=False)
    dl = DataLoader(ds, batch_size=batch_size,
                    shuffle=False, num_workers=0)
    model.eval()
    corr = tot = 0
    all_p, all_l = [], []
    with torch.no_grad():
        for R,K,labels in dl:
            R, K, labels = R.to(device), K.to(device), labels.to(device)
            out = model(R,K)
            preds = out.argmax(dim=1)
            corr += (preds == labels).sum().item()
            tot  += labels.size(0)
            all_p += preds.cpu().tolist()
            all_l += labels.cpu().tolist()

    acc = 100 * corr / tot
    print(f"Test Acc: {acc:.2f}%")
    print("CM:\n", confusion_matrix(all_l, all_p))
    print("Report:\n", classification_report(all_l, all_p,
                                             target_names=actions))

# =============================================================================
# 8. MAIN
# =============================================================================
if __name__=="__main__":
    actions       = ["Death","Doctor","he","healthy","Hospital",
                     "I","Medicine","Patient","she","sick"]
    raw_videos_dir = "./dataset"
    kp_dir         = "./extracted_keypoints"
    keypoint_dim = 1662
    num_classes  = len(actions)
    model = FusedModel(keypoint_dim, num_classes).to(device)
    model.load_state_dict(torch.load("fused_model29.pth", map_location=device))

    # run
    test_video = "./videos/sickOGdataset.MOV"
    evaluate_model(model, raw_videos_dir, kp_dir, actions,
                   seq_len=60, batch_size=8)

Test Acc: 93.08%
CM:
 [[29  0  0  0  0  0  0  0  0  0]
 [ 0 29  0  0  0  0  0  0  0  0]
 [ 0  0 26  0  0  2  0  0  1  0]
 [ 0  0  0 29  0  0  0  0  0  0]
 [ 0  0  0  0 29  0  0  0  0  0]
 [ 0  0  0  0  0 29  0  0  0  0]
 [ 3  0  0  0  0  0 26  0  0  0]
 [ 2  6  0  0  0  0  0 20  0  0]
 [ 0  0  2  0  0  4  0  0 23  0]
 [ 0  0  0  0  0  0  0  0  0 29]]
Report:
               precision    recall  f1-score   support

       Death       0.85      1.00      0.92        29
      Doctor       0.83      1.00      0.91        29
          he       0.93      0.90      0.91        29
     healthy       1.00      1.00      1.00        29
    Hospital       1.00      1.00      1.00        29
           I       0.83      1.00      0.91        29
    Medicine       1.00      0.90      0.95        29
     Patient       1.00      0.71      0.83        28
         she       0.96      0.79      0.87        29
        sick       1.00      1.00      1.00        29

    accuracy                           0.9

In [ ]:
# (Resnet + Fnet + LSTM) Sliding window AND SINGLE-SHOT test_fused_isl.py (Resnet + Fnet + LSTM)

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from collections import deque
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models.feature_extraction import create_feature_extractor
import mediapipe as mp

# =============================================================================
# 1) DEVICE & RESNET FEATURIZER
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

weights = ResNet18_Weights.IMAGENET1K_V1
_resnet = resnet18(weights=weights).to(device).eval()
fe = create_feature_extractor(_resnet, return_nodes=['avgpool'])

# =============================================================================
# 2) MEDIAPIPE LANDMARK EXTRACTION
# =============================================================================
def extract_all_keypoints(results):
    # (copy your full normalization from training)
    # returns a flat numpy vector of length ~1662
    # — pose (33×4), face (468×3), left hand (21×3), right hand (21×3)
    if results.pose_landmarks:
        pose = np.array([[l.x,l.y,l.z,l.visibility]
                         for l in results.pose_landmarks.landmark])
        try:
            a,b = pose[11,:2], pose[12,:2]
            center = (a+b)/2; scale = np.linalg.norm(a-b) or 1.0
        except:
            center,scale = np.array([0.5,0.5]),1.0
        norm_pose = np.hstack([ (pose[:,:2]-center)/scale,
                                pose[:,2:3], pose[:,3:4] ])
    else:
        norm_pose = np.zeros((33,4))
    if results.face_landmarks:
        face = np.array([[l.x,l.y,l.z]
                         for l in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face[:,:2]-center)/scale,
                                face[:,2:3] ])
    else:
        norm_face = np.zeros((468,3))
    if results.left_hand_landmarks:
        lh = np.array([[l.x,l.y,l.z]
                       for l in results.left_hand_landmarks.landmark])
        w = lh[0,:2]
        norm_lh = np.hstack([ (lh[:,:2]-w)/scale, lh[:,2:3] ])
    else:
        norm_lh = np.zeros((21,3))
    if results.right_hand_landmarks:
        rh = np.array([[l.x,l.y,l.z]
                       for l in results.right_hand_landmarks.landmark])
        w = rh[0,:2]
        norm_rh = np.hstack([ (rh[:,:2]-w)/scale, rh[:,2:3] ])
    else:
        norm_rh = np.zeros((21,3))
    return np.concatenate([
        norm_pose.flatten(),
        norm_face.flatten(),
        norm_lh.flatten(),
        norm_rh.flatten()
    ])

# =============================================================================
# 3) FRAME SAMPLING UTILITY
# =============================================================================
def sample_frames(video_path, N=60):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frm = cap.read()
        if not ret: break
        frames.append(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    cap.release()
    if len(frames)==0:
        raise ValueError(f"No frames in {video_path}")
    idxs = np.linspace(0, len(frames)-1, N).round().astype(int)
    return [frames[i] for i in idxs]

# =============================================================================
# 4) FNet BLOCK & FUSED MODEL DEFINITION
# =============================================================================
class FNetBlock(nn.Module):
    def forward(self, x):
        xf = torch.fft.rfft(x, dim=1)
        return torch.fft.irfft(xf, n=x.size(1), dim=1)

class FusedModel(nn.Module):
    def __init__(self, keypoint_dim, num_classes, lstm_hidden=256):
        super().__init__()
        self.resnet_proj   = nn.Linear(512,256)
        self.landmark_proj = nn.Linear(keypoint_dim,256)
        self.relu          = nn.ReLU()
        self.fnet          = FNetBlock()
        self.lstm          = nn.LSTM(512, lstm_hidden,
                                     batch_first=True,
                                     bidirectional=True)
        self.classifier    = nn.Linear(lstm_hidden*2, num_classes)

    def forward(self, R, K):
        # R: (B,60,512), K: (B,60,keypoint_dim)
        Rp = self.relu(self.resnet_proj(R))
        Kp = self.relu(self.landmark_proj(K))
        x  = torch.cat([Rp,Kp], dim=-1)          # (B,60,512)
        x, _ = self.lstm(self.fnet(x))           # (B,60,512)
        return self.classifier(x[:,-1,:])        # (B,num_classes)

# =============================================================================
# 5) SINGLE‐SHOT EVALUATION
# =============================================================================
def evaluate_once(video_path, model, actions):
    """ Sample 60 frames → extract ResNet + landmarks → one model forward """
    basename = os.path.splitext(os.path.basename(video_path))[0]
    # 1) ResNet features
    imgs = sample_frames(video_path, 60)
    feats = []
    with torch.no_grad():
        for img in imgs:
            t = transform(img).unsqueeze(0).to(device)
            out = fe(t)['avgpool'].view(-1).cpu().numpy()
            feats.append(out)
    R_seq = torch.tensor(np.stack(feats),
                         dtype=torch.float32).unsqueeze(0).to(device)

    # 2) Mediapipe landmarks
    holo = mp.solutions.holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)
    kpts = []
    for img in imgs:
        res = holo.process(img)
        kpts.append(extract_all_keypoints(res))
    holo.close()
    K_seq = torch.tensor(np.stack(kpts),
                         dtype=torch.float32).unsqueeze(0).to(device)

    # 3) Forward + print
    model.eval()
    with torch.no_grad():
        logits = model(R_seq, K_seq)
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
    idx = int(probs.argmax())
    print(f">>> Single‐shot: {actions[idx]}  (conf={probs[idx]:.2f})")
    print({actions[i]:float(probs[i]) for i in range(len(actions))})

# =============================================================================
# 6) SLIDING‐WINDOW “LIVE” PREVIEW
# =============================================================================
def sliding_window(video_path, model, actions,
                   window_size=60, smooth_w=10, thresh=0.7):
    """ Real‐time loop: accumulate 60 frames + keypoints, infer + smooth """
    cap = cv2.VideoCapture(video_path)
    holo = mp.solutions.holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)
    Rwin, Kwin, preds = deque(maxlen=window_size), deque(maxlen=window_size), deque(maxlen=smooth_w)
    sent = []
    while True:
        ret, frame = cap.read()
        if not ret: 
            # Loop the video if it ends
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # ResNet
        with torch.no_grad():
            t = transform(rgb).unsqueeze(0).to(device)
            r = fe(t)['avgpool'].view(-1).cpu().numpy()
        Rwin.append(r)
        # Keypoints
        res = holo.process(rgb)
        Kwin.append(extract_all_keypoints(res))

        # infer when full
        if len(Rwin)==window_size:
            Rb = torch.tensor(np.stack(Rwin),
                              dtype=torch.float32).unsqueeze(0).to(device)
            Kb = torch.tensor(np.stack(Kwin),
                              dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                out  = model(Rb,Kb)
                prob = torch.softmax(out, dim=1)[0].cpu().numpy()
            pidx = int(prob.argmax()); preds.append(pidx)
            if len(preds)==smooth_w and len(set(preds))==1 and prob[pidx]>=thresh:
                w = actions[pidx]
                if not sent or sent[-1]!=w:
                    sent.append(w)

        # overlay
        disp = frame.copy()
        cv2.putText(disp, " ".join(sent), (10,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)
        cv2.imshow("ISL Live", disp)
        if cv2.waitKey(1)&0xFF==ord('q'): break

    cap.release()
    holo.close()
    cv2.destroyAllWindows()

# =============================================================================
# 7) RUN IT!
# =============================================================================
if __name__=="__main__":
    ACTIONS = ["Death","Doctor","he","healthy","Hospital",
               "I","Medicine","Patient","she","sick"]
    # load your model
    kp_dim = 33*4 + 468*3 + 21*3 + 21*3
    model = FusedModel(kp_dim, len(ACTIONS)).to(device)
    model.load_state_dict(torch.load("fused_model29.pth",
                                     map_location=device))

    TEST_VIDEO = "./videos/SourikSampleVideo.mp4"

    # single‐shot
    #evaluate_once(TEST_VIDEO, model, ACTIONS)

    # real‐time sliding window
    sliding_window(TEST_VIDEO, model, ACTIONS)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

In [7]:
#Evaluate for ResNet

def predict_video(model, data_dir, video_path, actions, seq_len=60):
    """
    Run inference on a single video file.
    
    Args:
      model      : Trained ResNetFNetLSTM instance.
      data_dir   : Path to your './dataset' root (for transforms & ResNet featurizer).
      video_path : Path to the raw .mp4/.avi/.mov file to predict.
      actions    : List of class names, e.g. ["Death","Doctor",…].
      seq_len    : Number of frames to sample (default=60).
    
    Returns:
      pred_idx, pred_probs : Index of the top class and the probability vector.
    """
    # 1. Recreate the Dataset head to get transform & ResNet feature extractor
    ds    = VideoResNetDataset(data_dir, actions, seq_len, augment=False)
    transform = ds.transform                        # ImageNet → tensor → normalize 
    fe       = ds.fe.to(device)                     # Frozen ResNet up to 'avgpool' 

    # 2. Sample frames
    frames = sample_frames(video_path, seq_len)      # Uniform 60‑frame sampling :contentReference[oaicite:2]{index=2}

    # 3. Extract per‑frame features
    feats = []
    with torch.no_grad():
        for img in frames:
            t  = transform(img).unsqueeze(0).to(device)
            out = fe(t)['avgpool']                   # (1,512,1,1)
            feats.append(out.view(-1))              # → (512,)

    # 4. Stack into sequence tensor
    seq = torch.stack(feats).unsqueeze(0).to(device) # (1, 60, 512)

    # 5. Forward through ResNet→FNet→BiLSTM→FC
    model.eval()
    with torch.no_grad():
        logits = model(seq)                          # (1, num_classes)
        probs  = torch.softmax(logits, dim=1).cpu().numpy()[0]

    # 6. Print results
    idx = probs.argmax()
    print(f"Predicted: {actions[idx]} (p={probs[idx]:.2f})")
    for act, p in zip(actions, probs):
        print(f"  {act:<8}: {p:.4f}")

    return idx, probs

# ------------------------------
# Usage example (after training):
# ------------------------------
if __name__ == "__main__":
    # Load your trained model
    model = ResNetFNetLSTM(feat_dim=512, num_classes=len(actions)).to(device)
    model.load_state_dict(torch.load("resnet_fnet_lstm.pth"))
    
    # Run prediction on a new video
    video_file = "./videos/SickSourikSampleVideo - Trim2.mp4"
    pred_idx, pred_probs = predict_video(
        model, data_dir="./dataset",
        video_path=video_file,
        actions=actions,
        seq_len=60
    )


Predicted: she (p=0.44)
  Death   : 0.0049
  Doctor  : 0.0241
  he      : 0.1845
  healthy : 0.0124
  Hospital: 0.0069
  I       : 0.2808
  Medicine: 0.0016
  Patient : 0.0085
  she     : 0.4359
  sick    : 0.0405


In [14]:
evaluate_model(model, extracted_keypoints_dir, actions, sequence_length=60, batch_size=32)

Test Accuracy: 91.43%
Confusion Matrix:
[[14  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  1  4  0]
 [ 0  0  0 14  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  1  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0  0 11  2  1]
 [ 0  0  2  0  0  1  0  0 11  0]
 [ 0  0  0  0  0  0  0  0  0 14]]
Classification Report:
              precision    recall  f1-score   support

       Death       1.00      1.00      1.00        14
      Doctor       1.00      1.00      1.00        14
          he       0.82      0.64      0.72        14
     healthy       1.00      1.00      1.00        14
    Hospital       1.00      1.00      1.00        14
           I       0.93      0.93      0.93        14
    Medicine       1.00      1.00      1.00        14
     Patient       0.92      0.79      0.85        14
         she       0.61      0.79      0.69        14
        sick       0.93      1.00      0.97        14

    accura

(91.42857142857143,
 array([[14,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 14,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  9,  0,  0,  0,  0,  1,  4,  0],
        [ 0,  0,  0, 14,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 14,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0, 13,  0,  0,  1,  0],
        [ 0,  0,  0,  0,  0,  0, 14,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, 11,  2,  1],
        [ 0,  0,  2,  0,  0,  1,  0,  0, 11,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 14]]),
 '              precision    recall  f1-score   support\n\n       Death       1.00      1.00      1.00        14\n      Doctor       1.00      1.00      1.00        14\n          he       0.82      0.64      0.72        14\n     healthy       1.00      1.00      1.00        14\n    Hospital       1.00      1.00      1.00        14\n           I       0.93      0.93      0.93        14\n    Medicine       1.00      1.00      1.00        14\n     Patient       0.92      0.79 

In [18]:
#Evaluate single video 60 frames

def evaluate_raw_video(model, video_file, sequence_length=60):
    """
    Evaluates a raw video file (without pre-extracted keypoints).
    This function:
      1. Reads the video and uniformly samples exactly 'sequence_length' frames.
      2. Processes each sampled frame with MediaPipe Holistic to extract normalized keypoints.
      3. Stacks the keypoints into a sequence.
      4. Feeds the sequence into the trained model to obtain a prediction.
      
    Returns:
      pred_idx: the predicted class index.
      probabilities: the softmax probabilities for each class.
    """
    # Open the video file.
    cap = cv2.VideoCapture(video_file)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()

    if len(all_frames) == 0:
        print(f"Error: No frames extracted from video {video_file}. Check the file path or video content.")
        return None, None

    # Uniformly sample the frames.
    sampled_frames = sample_frames_uniformly(all_frames, target_count=sequence_length)
    
    # Process each frame using MediaPipe to extract normalized keypoints.
    keypoint_list = []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frm in sampled_frames:
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            # Optionally, you can check if keypoints are all zeros and skip such frames.
            if np.all(keypoints == 0):
                print("Warning: No landmarks detected in one frame; using zeros for this frame.")
            keypoint_list.append(keypoints)
    
    if len(keypoint_list) == 0:
        print("Error: No keypoints extracted from the sampled frames.")
        return None, None
    
    try:
        sequence = np.stack(keypoint_list, axis=0)  # Shape: (sequence_length, keypoint_dim)
    except Exception as e:
        print("Error stacking keypoints:", e)
        return None, None
    
    # Convert to a tensor and add batch dimension.
    sample_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Evaluate using the trained model.
    model.eval()
    with torch.no_grad():
        outputs = model(sample_tensor)
        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()[0]
        pred_idx = np.argmax(probabilities)
    
    return pred_idx, probabilities

# =============================================================================
# Example usage in the main block:
# =============================================================================
if __name__ == "__main__":
    # Define your action classes.
    actions = [
    "Death",
    "Doctor",
    "he",
    "healthy",
    "Hospital",
    "I",
    "Medicine",
    "Patient",
    "she",
    "sick"
    ]

    # Load the trained model (adjust input_size if needed)
    input_size = 1662  # Or your normalized keypoint vector length.
    num_classes = len(actions)
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load("action_detection_model60_2.pth", map_location=device))
    
    # Evaluate a raw video sample.
    raw_video_file = "./videos/SickSourikSampleVideo - Trim2.mp4"  # Ensure this path is correct.
    pred_idx, probabilities = evaluate_raw_video(model, raw_video_file, sequence_length=60)
    if pred_idx is not None:
        print("Raw Video Evaluation:")
        print("Predicted Class:", actions[pred_idx])
        print("Probabilities:", probabilities)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Raw Video Evaluation:
Predicted Class: Patient
Probabilities: [2.9630264e-08 1.0585179e-04 1.0644831e-03 6.8382360e-07 1.9386948e-06
 6.0008224e-03 1.8744953e-05 9.6890914e-01 2.3895640e-02 2.6620478e-06]


In [13]:
# Evaluate the trained model on the test data
evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32)

# Run real-time sliding window prediction (adjust the video file path as needed)
test_video_file = "./videos/sick2.MOV"
#sliding_window_prediction(test_video_file, model, actions, window_size=60, smoothing_window=0, threshold=0.5)

NameError: name 'data_dir' is not defined

In [7]:
# ONLY Evaluate newly trained 60fps model with dropout and random variations in test data
# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    """
    Extract and normalize keypoints from MediaPipe detection results.
    Normalization: Uses the midpoint of the left and right shoulders (if available)
    as the center, and the shoulder distance as the scale factor.
    Returns a concatenated numpy array of normalized pose, face, left hand, and right hand keypoints.
    """
    # Normalize pose keypoints
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            # Landmarks 11 and 12 correspond to left and right shoulders
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        # Only normalize x and y coordinates; leave z and visibility unchanged
        norm_pose = np.hstack([ (pose_points[:,0:2] - center) / scale, pose_points[:,2:3], pose_points[:,3:4] ])
    else:
        norm_pose = np.zeros((33, 4))
    
    # Normalize face keypoints using the same center and scale
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face_points[:,0:2] - center) / scale, face_points[:,2:3] ])
    else:
        norm_face = np.zeros((468, 3))
    
    # Normalize left hand keypoints (using the wrist as the anchor)
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]  # assume index 0 is the wrist
        norm_left_hand = np.hstack([ (left_hand_points[:,0:2] - wrist) / scale, left_hand_points[:,2:3] ])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    # Normalize right hand keypoints (using the wrist as the anchor)
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([ (right_hand_points[:,0:2] - wrist) / scale, right_hand_points[:,2:3] ])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    """
    Apply augmentation to a keypoint sequence by adding Gaussian noise.
    Input:
      sequence: numpy array of shape (sequence_length, keypoint_dim)
    Returns an augmented sequence.
    """
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    """
    Given a list of frames, return exactly 'target_count' frames by uniformly sampling.
    """
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    """
    Process a video: sample frames, extract normalized keypoints using MediaPipe Holistic,
    and save each frame's keypoints as a separate .npy file.
    """
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS WITH AUGMENTATION OPTION
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    """
    Custom dataset for loading keypoint sequences.
    Expects data organized as:
      ./extracted_keypoints/<action>/<video_basename>/frame_*.npy
    """
    def __init__(self, data_dir, actions, sequence_length=60, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            for video_folder in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_folder)
                if not os.path.isdir(video_path):
                    continue
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    frames = []
                    for f in frame_files:
                        fpath = os.path.join(video_path, f)
                        frames.append(np.load(fpath))
                    sequence = np.stack(frames, axis=0)  # (sequence_length, keypoint_dim)
                    self.data.append(sequence)
                    self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if self.augment:
            sequence = augment_sequence(sequence)
        sample = torch.tensor(sequence, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    """
    3-layer LSTM architecture with dropout regularization and two dense layers.
    Architecture:
      LSTM(64) -> ReLU -> Dropout ->
      LSTM(128) -> ReLU -> Dropout ->
      LSTM(256) -> ReLU -> Dropout ->
      Dense(64) -> ReLU -> Dropout ->
      Dense(num_classes)
    """
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        # Use the output from the last time step
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    """
    Train the LSTM model. Optionally applies data augmentation.
    """
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    sample, _ = dataset[0]
    input_size = sample.shape[1]
    num_classes = len(actions)
    
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        elapsed = time.time() - start_time
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    
    torch.save(model.state_dict(), "action_detection_model60.pth")
    print("Model weights saved as 'action_detection_model60.pth'")
    return model

# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# 9. REAL-TIME SLIDING WINDOW PREDICTION
# =============================================================================
def sliding_window_prediction(video_file, model, actions, window_size=60, smoothing_window=10, threshold=0.5):
    """
    Processes a video file using a sliding window for real-time prediction.
    Applies a smoothing mechanism over recent predictions.
    """
    colors = [(245,117,16), (117,245,16), (16,117,245), (0,255,0)]
    sequence = []
    predictions = []
    sentence = []
    
    cap = cv2.VideoCapture(video_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            keypoints = extract_all_keypoints(results)
            sequence.append(keypoints)
            if len(sequence) > window_size:
                sequence = sequence[-window_size:]
            
            if len(sequence) == window_size:
                seq_array = np.stack(sequence, axis=0)
                seq_tensor = torch.tensor(seq_array, dtype=torch.float32).unsqueeze(0).to(device)
                model.eval()
                with torch.no_grad():
                    outputs = model(seq_tensor)
                    prob = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                    pred_idx = np.argmax(prob)
                    predictions.append(pred_idx)
                
                if len(predictions) >= smoothing_window:
                    recent_preds = predictions[-smoothing_window:]
                    if np.unique(recent_preds).size == 1 and prob[pred_idx] > threshold:
                        if len(sentence) == 0 or actions[pred_idx] != sentence[-1]:
                            sentence.append(actions[pred_idx])
                
                vis_frame = prob_viz(prob, actions, image, colors)
                cv2.rectangle(vis_frame, (0,0), (640,40), (245,117,16), -1)
                cv2.putText(vis_frame, ' '.join(sentence), (3,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Sliding Window Action Detection", vis_frame)
            else:
                cv2.imshow("Sliding Window Action Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# =============================================================================
# 10. PROBABILITY VISUALIZATION FUNCTION
# =============================================================================
def prob_viz(probabilities, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(probabilities):
        width = int(prob * 200)
        top_left = (0, 60 + num * 40)
        bottom_right = (width, 90 + num * 40)
        cv2.rectangle(output_frame, top_left, bottom_right, colors[num], -1)
        cv2.putText(output_frame, f"{actions[num]}: {prob:.2f}", (10, 85 + num * 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

class ActionModelPyTorch(nn.Module):
    """
    3-layer LSTM architecture with dropout regularization and two dense layers.
    Architecture:
      LSTM(64) -> ReLU -> Dropout ->
      LSTM(128) -> ReLU -> Dropout ->
      LSTM(256) -> ReLU -> Dropout ->
      Dense(64) -> ReLU -> Dropout ->
      Dense(num_classes)
    """
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        # Use the output from the last time step
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

def evaluate_raw_video(model, video_file, sequence_length=60):
    """
    Evaluates a raw video file (without pre-extracted keypoints).
    This function:
      1. Reads the video and uniformly samples exactly 'sequence_length' frames.
      2. Processes each sampled frame with MediaPipe Holistic to extract normalized keypoints.
      3. Stacks the keypoints into a sequence.
      4. Feeds the sequence into the trained model to obtain a prediction.
      
    Returns:
      pred_idx: the predicted class index.
      probabilities: the softmax probabilities for each class.
    """
    # Open the video file.
    cap = cv2.VideoCapture(video_file)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()

    # Uniformly sample the frames.
    sampled_frames = sample_frames_uniformly(all_frames, target_count=sequence_length)
    
    # Process each frame using MediaPipe to extract normalized keypoints.
    keypoint_list = []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frm in sampled_frames:
            # Convert the frame color and process it.
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            # Extract normalized keypoints.
            keypoints = extract_all_keypoints(results)
            keypoint_list.append(keypoints)
    
    # Stack keypoints to form a sequence.
    sequence = np.stack(keypoint_list, axis=0)  # Shape: (sequence_length, keypoint_dim)
    
    # Convert to a tensor and add batch dimension.
    sample_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Evaluate using the trained model.
    model.eval()
    with torch.no_grad():
        outputs = model(sample_tensor)
        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()[0]
        pred_idx = np.argmax(probabilities)
    
    return pred_idx, probabilities

# =============================================================================
# Example Usage:
# =============================================================================
if __name__ == "__main__":
    # Define your action classes.
    actions = [
    "Death",
    "Doctor",
    "he",
    "healthy",
    "Hospital",
    "I",
    "Medicine",
    "Patient",
    "she",
    "sick"
    ]
    
    # Define the path to your raw video file (without extracted keypoints).
    raw_video_file = "./videos/SickSourikSampleVideo - Trim2.mp4"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Load the trained model.
    input_size = 1662  # Adjust if needed based on your keypoint vector length.
    num_classes = len(actions)
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load("action_detection_model60_29.pth", map_location=device))
    
    # Evaluate the raw video.
    pred_idx, probabilities = evaluate_raw_video(model, raw_video_file, sequence_length=60)
    
    if pred_idx is not None:
        print("Predicted Class:", actions[pred_idx])
        print("Probabilities:", probabilities)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Predicted Class: he
Probabilities: [0.09698344 0.00827818 0.6284523  0.00385565 0.04178295 0.10454544
 0.00696367 0.01597067 0.09062362 0.00254412]


In [ ]:
#Preview + Evaluate newly trained 60fps model with dropout and random variations in test data

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    """
    Extract and normalize keypoints from MediaPipe detection results.
    Normalization: Uses the midpoint of the left and right shoulders (if available)
    as the center, and the shoulder distance as the scale factor.
    Returns a concatenated numpy array of normalized pose, face, left hand, and right hand keypoints.
    """
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        norm_pose = np.hstack([ (pose_points[:,0:2] - center) / scale, pose_points[:,2:3], pose_points[:,3:4] ])
    else:
        norm_pose = np.zeros((33, 4))
    
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face_points[:,0:2] - center) / scale, face_points[:,2:3] ])
    else:
        norm_face = np.zeros((468, 3))
    
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]
        norm_left_hand = np.hstack([ (left_hand_points[:,0:2] - wrist) / scale, left_hand_points[:,2:3] ])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([ (right_hand_points[:,0:2] - wrist) / scale, right_hand_points[:,2:3] ])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS WITH AUGMENTATION OPTION
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    def __init__(self, data_dir, actions, sequence_length=60, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            for video_folder in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_folder)
                if not os.path.isdir(video_path):
                    continue
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    frames = []
                    for f in frame_files:
                        fpath = os.path.join(video_path, f)
                        frames.append(np.load(fpath))
                    sequence = np.stack(frames, axis=0)
                    self.data.append(sequence)
                    self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if self.augment:
            sequence = augment_sequence(sequence)
        sample = torch.tensor(sequence, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    sample, _ = dataset[0]
    input_size = sample.shape[1]
    num_classes = len(actions)
    
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        elapsed = time.time() - start_time
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    
    torch.save(model.state_dict(), "action_detection_model60.pth")
    print("Model weights saved as 'action_detection_model60.pth'")
    return model

# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# 9. REAL-TIME SLIDING WINDOW PREDICTION WITH PROBABILITY VISUALISATION
# =============================================================================
def prob_viz(probabilities, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(probabilities):
        width = int(prob * 200)
        top_left = (0, 60 + num * 40)
        bottom_right = (width, 90 + num * 40)
        cv2.rectangle(output_frame, top_left, bottom_right, colors[num], -1)
        cv2.putText(output_frame, f"{actions[num]}: {prob:.2f}", (10, 85 + num * 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

def sliding_window_prediction(video_file, model, actions, window_size=60, smoothing_window=10, threshold=0.5):
    """
    Processes a video file using a sliding window for real-time prediction.
    The video preview shows the probability visualization overlay and loops until 'q' is pressed.
    Predictions are updated continuously throughout the video.
    """
    colors = [(245,117,16), (117,245,16), (16,117,245), (0,255,0)]
    sequence = []
    predictions = []
    sentence = []
    
    cap = cv2.VideoCapture(video_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            # Process frame and extract keypoints
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            keypoints = extract_all_keypoints(results)
            sequence.append(keypoints)
            if len(sequence) > window_size:
                sequence = sequence[-window_size:]
            
            if len(sequence) == window_size:
                seq_array = np.stack(sequence, axis=0)
                seq_tensor = torch.tensor(seq_array, dtype=torch.float32).unsqueeze(0).to(device)
                model.eval()
                with torch.no_grad():
                    outputs = model(seq_tensor)
                    prob = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                    pred_idx = np.argmax(prob)
                    predictions.append(pred_idx)
                
                if len(predictions) >= smoothing_window:
                    recent_preds = predictions[-smoothing_window:]
                    if np.unique(recent_preds).size == 1 and prob[pred_idx] > threshold:
                        if len(sentence) == 0 or actions[pred_idx] != sentence[-1]:
                            sentence.append(actions[pred_idx])
                
                # Overlay probability visualization and prediction sentence on the frame.
                vis_frame = prob_viz(prob, actions, image, colors)
                cv2.rectangle(vis_frame, (0,0), (640,40), (245,117,16), -1)
                cv2.putText(vis_frame, ' '.join(sentence), (3,30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Sliding Window Action Detection", vis_frame)
            else:
                cv2.imshow("Sliding Window Action Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# =============================================================================
# 10. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    # Define action classes and directories
    actions = actions = [
    "Baby", "Bill", "Boy", "Brother", "Daughter", "Death",
    "Doctor", "Family", "Father", "Girl", "he", "healthy",
    "Hospital", "Husband", "I", "it", "Man", "Medicine",
    "Money", "Mother", "Parent", "Patient", "Price",
    "she", "sick", "Sign", "Sister", "Son", "Student",
    "they", "we", "Wife", "Woman", "you", "you (plural)"
    ]
    extracted_keypoints_dir = "./extracted_keypoints"  # Folder with pre-extracted keypoints
    raw_dataset_dir = "./dataset"  # Folder with raw training videos (organized by action)
    
    # Uncomment the following block to extract keypoints from raw training videos.
    # for action in actions:
    #     action_raw_folder = os.path.join(raw_dataset_dir, action)
    #     action_extracted_folder = os.path.join(extracted_keypoints_dir, action)
    #     os.makedirs(action_extracted_folder, exist_ok=True)
    #     video_files = [vf for vf in os.listdir(action_raw_folder) if vf.lower().endswith((".mp4", ".avi", ".mov"))]
    #     for video_file in video_files:
    #         video_path = os.path.join(action_raw_folder, video_file)
    #         video_basename = os.path.splitext(video_file)[0]
    #         video_output_folder = os.path.join(action_extracted_folder, video_basename)
    #         os.makedirs(video_output_folder, exist_ok=True)
    #         print(f"Processing video: {video_path}")
    #         process_video_and_save_frames(video_path, video_output_folder, target_frames=60)
    
    # Load the trained model.
    input_size = 1662  # Adjust if needed based on your keypoint vector length.
    num_classes = len(actions)
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load("action_detection_model60.pth", map_location=device))
    # ---------------------------------------------
    # Sliding Window Prediction on a Long Test Video
    # ---------------------------------------------
    # Path to a raw testing video (can be longer than 60 frames)
    test_video_file = "./videos/SickSourikSampleVideo - Trim2.mp4"  # Adjust the path as needed
    sliding_window_prediction(test_video_file, model, actions, window_size=60, smoothing_window=10, threshold=0.5)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

In [11]:
#With Confidence Thresholding + Preview + Evaluate newly trained 60fps model with dropout and random variations in test data

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    """
    Extract and normalize keypoints from MediaPipe detection results.
    Normalization: Uses the midpoint of the left and right shoulders (if available)
    as the center, and the shoulder distance as the scale factor.
    Returns a concatenated numpy array of normalized pose, face, left hand, and right hand keypoints.
    """
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        norm_pose = np.hstack([ (pose_points[:,0:2] - center) / scale, pose_points[:,2:3], pose_points[:,3:4] ])
    else:
        norm_pose = np.zeros((33, 4))
    
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face_points[:,0:2] - center) / scale, face_points[:,2:3] ])
    else:
        norm_face = np.zeros((468, 3))
    
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]
        norm_left_hand = np.hstack([ (left_hand_points[:,0:2] - wrist) / scale, left_hand_points[:,2:3] ])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([ (right_hand_points[:,0:2] - wrist) / scale, right_hand_points[:,2:3] ])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS WITH AUGMENTATION OPTION
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    def __init__(self, data_dir, actions, sequence_length=60, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            for video_folder in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_folder)
                if not os.path.isdir(video_path):
                    continue
                frame_files = [f for f in os.listdir(video_path) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    frames = []
                    for f in frame_files:
                        fpath = os.path.join(video_path, f)
                        frames.append(np.load(fpath))
                    sequence = np.stack(frames, axis=0)
                    self.data.append(sequence)
                    self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if self.augment:
            sequence = augment_sequence(sequence)
        sample = torch.tensor(sequence, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    sample, _ = dataset[0]
    input_size = sample.shape[1]
    num_classes = len(actions)
    
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        elapsed = time.time() - start_time
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    
    torch.save(model.state_dict(), "action_detection_model60.pth")
    print("Model weights saved as 'action_detection_model60.pth'")
    return model

# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# 9. REAL-TIME SLIDING WINDOW PREDICTION WITH PROBABILITY VISUALISATION
# =============================================================================
def prob_viz(probabilities, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(probabilities):
        width = int(prob * 200)
        top_left = (0, 60 + num * 40)
        bottom_right = (width, 90 + num * 40)
        cv2.rectangle(output_frame, top_left, bottom_right, colors[num], -1)
        cv2.putText(output_frame, f"{actions[num]}: {prob:.2f}", (10, 85 + num * 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

def sliding_window_prediction(video_file, model, actions, window_size=60, smoothing_window=20, threshold=0.7):
    """
    Processes a video file using a sliding window for real-time prediction.
    Applies per-frame confidence thresholding and a temporal consistency check:
      - For each window of 'window_size' frames, the model outputs a prediction.
      - Only if the last 'smoothing_window' predictions are for the same class and each has
        a confidence (softmax score) above 'threshold' is that prediction accepted.
      - Otherwise, "NA" is shown.
    The video preview shows the probability visualization overlay and loops until 'q' is pressed.
    """
    colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (0, 255, 0),
          (255, 0, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
          (0, 255, 255), (128, 128, 128)]
    sequence = []  # Store keypoints for the current sliding window
    predictions = []  # List of predicted class indices for recent windows
    prediction_confidences = []  # List of corresponding confidence scores
    displayed_prediction = "NA"  # Currently registered prediction (default: NA)
    
    cap = cv2.VideoCapture(video_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                # Loop the video if it ends
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            # Process frame and extract keypoints
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            keypoints = extract_all_keypoints(results)
            sequence.append(keypoints)
            if len(sequence) > window_size:
                sequence = sequence[-window_size:]
            
            if len(sequence) == window_size:
                seq_array = np.stack(sequence, axis=0)  # Shape: (window_size, keypoint_dim)
                seq_tensor = torch.tensor(seq_array, dtype=torch.float32).unsqueeze(0).to(device)
                
                model.eval()
                with torch.no_grad():
                    outputs = model(seq_tensor)
                    prob = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                    pred_idx = np.argmax(prob)
                    conf = prob[pred_idx]
                    predictions.append(pred_idx)
                    prediction_confidences.append(conf)
                
                # Check if we have enough recent predictions for temporal consistency
                if len(prediction_confidences) >= smoothing_window:
                    recent_preds = predictions[-smoothing_window:]
                    recent_confs = prediction_confidences[-smoothing_window:]
                    # Temporal condition: All recent predictions are for the same class
                    # and each has a confidence above the threshold.
                    if (len(set(recent_preds)) == 1) and all(c >= threshold for c in recent_confs):
                        current_prediction = actions[pred_idx]
                    else:
                        current_prediction = "NA"
                    
                    # Update the displayed prediction only if it has changed.
                    if current_prediction != displayed_prediction:
                        displayed_prediction = current_prediction
                
                # Overlay the probability visualization and the current prediction
                vis_frame = prob_viz(prob, actions, image, colors)
                cv2.rectangle(vis_frame, (0, 0), (640, 40), (245,117,16), -1)
                cv2.putText(vis_frame, displayed_prediction, (3, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Sliding Window Action Detection", vis_frame)
            else:
                cv2.imshow("Sliding Window Action Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()


# =============================================================================
# 10. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    # Define action classes and directories
    actions = [
    "Death",
    "Doctor",
    "he",
    "healthy",
    "Hospital",
    "I",
    "Medicine",
    "Patient",
    "she",
    "sick"
    ]
    extracted_keypoints_dir = "./extracted_keypoints"  # Folder with pre-extracted keypoints
    raw_dataset_dir = "./dataset"  # Folder with raw training videos (organized by action)
    
    # Uncomment the following block to extract keypoints from raw training videos.
    # for action in actions:
    #     action_raw_folder = os.path.join(raw_dataset_dir, action)
    #     action_extracted_folder = os.path.join(extracted_keypoints_dir, action)
    #     os.makedirs(action_extracted_folder, exist_ok=True)
    #     video_files = [vf for vf in os.listdir(action_raw_folder) if vf.lower().endswith((".mp4", ".avi", ".mov"))]
    #     for video_file in video_files:
    #         video_path = os.path.join(action_raw_folder, video_file)
    #         video_basename = os.path.splitext(video_file)[0]
    #         video_output_folder = os.path.join(action_extracted_folder, video_basename)
    #         os.makedirs(video_output_folder, exist_ok=True)
    #         print(f"Processing video: {video_path}")
    #         process_video_and_save_frames(video_path, video_output_folder, target_frames=60)
    
    # Load the trained model.
    input_size = 1662  # Adjust if needed based on your keypoint vector length.
    num_classes = len(actions)
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load("action_detection_model60_29.pth", map_location=device))
    # ---------------------------------------------
    # Sliding Window Prediction on a Long Test Video
    # ---------------------------------------------
    # Path to a raw testing video (can be longer than 60 frames)
    test_video_file = "./videos/SheSickMayukh.mp4"  # Adjust the path as needed
    sliding_window_prediction(test_video_file, model, actions, window_size=60, smoothing_window=10, threshold=0.5)


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

In [8]:
#!pip install torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.4 MB/s eta 0:00:00


In [10]:
#!pip install pytorch-tcn

In [ ]:
# TCN + Hybrid model

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
import mediapipe as mp

# =============================================================================
# 1) DEVICE & IMAGE TRANSFORMS
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# =============================================================================
# 2) MEDIAPIPE KEYPOINT EXTRACTION
# =============================================================================
mp_holistic = mp.solutions.holistic
def extract_all_keypoints(results):
    if results.pose_landmarks:
        pose = np.array([[l.x,l.y,l.z,l.visibility]
                         for l in results.pose_landmarks.landmark])
        a,b = pose[11,:2], pose[12,:2]
        center, scale = (a+b)/2, np.linalg.norm(a-b) or 1.0
        norm_pose = np.hstack([ (pose[:,:2]-center)/scale,
                                pose[:,2:3], pose[:,3:4] ])
    else:
        norm_pose = np.zeros((33,4))
    if results.face_landmarks:
        face = np.array([[l.x,l.y,l.z]
                         for l in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face[:,:2]-center)/scale,
                                face[:,2:3] ])
    else:
        norm_face = np.zeros((468,3))
    if results.left_hand_landmarks:
        lh = np.array([[l.x,l.y,l.z]
                       for l in results.left_hand_landmarks.landmark])
        wrist = lh[0,:2]
        norm_lh = np.hstack([ (lh[:,:2]-wrist)/scale, lh[:,2:3] ])
    else:
        norm_lh = np.zeros((21,3))
    if results.right_hand_landmarks:
        rh = np.array([[l.x,l.y,l.z]
                       for l in results.right_hand_landmarks.landmark])
        wrist = rh[0,:2]
        norm_rh = np.hstack([ (rh[:,:2]-wrist)/scale, rh[:,2:3] ])
    else:
        norm_rh = np.zeros((21,3))
    return np.concatenate([
        norm_pose.flatten(),
        norm_face.flatten(),
        norm_lh.flatten(),
        norm_rh.flatten()
    ])

# =============================================================================
# 3) KEYPOINT AUGMENTATION
# =============================================================================
class KeypointAugmentor:
    def __init__(self, noise_std=0.01, time_warp_sigma=0.2):
        self.noise_std = noise_std
        self.time_warp_sigma = time_warp_sigma
    def __call__(self, seq):
        # seq: (T, D)
        T_steps = seq.shape[0]
        window = np.exp(-0.5*(np.linspace(-2,2,T_steps)**2)/self.time_warp_sigma**2)
        seq = seq * torch.from_numpy(window).unsqueeze(1).to(seq.dtype)
        return seq + torch.randn_like(seq) * self.noise_std

# =============================================================================
# 4) FRAME SAMPLING
# =============================================================================
def sample_frames(video_path, N=60):
    cap, frames = cv2.VideoCapture(video_path), []
    while True:
        ret, frm = cap.read()
        if not ret: break
        frames.append(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    cap.release()
    if not frames:
        raise RuntimeError(f"No frames in {video_path}")
    idxs = np.linspace(0, len(frames)-1, N).round().astype(int)
    return [frames[i] for i in idxs]

# =============================================================================
# 5) DATASET USING FULL DATA
# =============================================================================
class FullDataset(Dataset):
    def __init__(self, video_dir, kp_dir, actions, seq_len=60, augment=False):
        self.samples, self.augment = [], augment
        self.kp_aug = KeypointAugmentor()
        for act in actions:
            act_vids = os.listdir(os.path.join(kp_dir, act))
            for vid in act_vids:
                # find video file
                for ext in ('.MP4','.mp4','.MOV','.mov'):
                    vp = os.path.join(video_dir, act, vid+ext)
                    if os.path.isfile(vp):
                        video_path = vp
                        break
                else:
                    raise FileNotFoundError(f"{vid}.* not found")
                # collect keypoints
                kpf = os.path.join(kp_dir, act, vid)
                files = sorted(os.listdir(kpf),
                               key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(files)!=seq_len: continue
                self.samples.append((video_path, kpf, actions.index(act)))
        self.seq_len = seq_len

    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        vp, kpf, lbl = self.samples[i]
        # frames → features
        imgs = sample_frames(vp, self.seq_len)
        frames = torch.stack([img_transform(img) for img in imgs], dim=0)
        # keypoints
        kps = []
        for f in sorted(os.listdir(kpf),
                        key=lambda x: int(x.split('_')[1].split('.')[0])):
            kps.append(np.load(os.path.join(kpf, f)))
        kps = torch.tensor(np.stack(kps), dtype=torch.float32)
        if self.augment:
            kps = self.kp_aug(kps)
        return frames, kps, lbl

# =============================================================================
# 6) TEMPORAL CONVOLUTIONAL NETWORK
# =============================================================================
class TCNBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, d=1):
        super().__init__()
        pad = (k-1)*d//2
        self.conv1 = nn.Conv1d(in_ch, out_ch, k, padding=pad, dilation=d)
        self.conv2 = nn.Conv1d(out_ch, out_ch, k, padding=pad, dilation=d)
        self.down = nn.Conv1d(in_ch, out_ch, 1) if in_ch!=out_ch else None
        self.relu = nn.ReLU()
    def forward(self, x):
        y = self.relu(self.conv1(x))
        y = self.conv2(y)
        res = x if self.down is None else self.down(x)
        return self.relu(y + res)

class TemporalConvNet(nn.Module):
    def __init__(self, in_ch, levels=4, k=3, ch=512):
        super().__init__()
        layers = []
        for i in range(levels):
            layers.append(TCNBlock(in_ch if i==0 else ch, ch, k, 2**i))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        # x: (B,T,C)
        x = x.transpose(1,2)    # → (B,C,T)
        y = self.net(x)
        return y.transpose(1,2) # → (B,T,C)

# =============================================================================
# 7) ATTENTION + CLASSIFIER
# =============================================================================
class Attention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(dim,1)
    def forward(self, x):
        # x: (B,T,dim)
        w = torch.softmax(self.fc(x), dim=1)
        return (w * x).sum(dim=1)  # (B,dim)

# =============================================================================
# 8) MULTI-MODAL MODEL
# =============================================================================
class ISLModel(nn.Module):
    def __init__(self, kp_dim, n_classes):
        super().__init__()
        self.resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Identity()
        for p in self.resnet.parameters(): p.requires_grad=False
        self.img_proj  = nn.Linear(512,256)
        self.kp_proj   = nn.Linear(kp_dim,256)
        self.fuse_fc   = nn.Linear(512,512)
        self.tcn       = TemporalConvNet(512,levels=4)
        self.attn      = Attention(512)
        self.classifier= nn.Linear(512,n_classes)
        self.relu      = nn.ReLU()

    def forward(self, imgs, kps):
        B,T = imgs.shape[0], imgs.shape[1]
        img_feats = []
        for t in range(T):
            f = self.resnet(imgs[:,t].to(device))
            img_feats.append(self.relu(self.img_proj(f)))
        img_feats = torch.stack(img_feats, dim=1)      # (B,T,256)
        kp_feats  = self.relu(self.kp_proj(kps.to(device)))  # (B,T,256)
        x = torch.cat([img_feats, kp_feats], dim=-1)   # (B,T,512)
        x = self.relu(self.fuse_fc(x))                 # (B,T,512)
        x = self.tcn(x)                                # (B,T,512)
        ctx = self.attn(x)                             # (B,512)
        return self.classifier(ctx)                    # (B,n_classes)

# =============================================================================
# 9) TRAIN ON FULL DATA WITH EARLY STOPPING
# =============================================================================
def train_full(video_dir, kp_dir, actions,
               seq_len=60, batch_size=8,
               epochs=500, lr=1e-3, early_stop_acc=0.92):
    # use entire dataset
    ds = FullDataset(video_dir, kp_dir, actions,
                     seq_len=seq_len, augment=True)
    dl = DataLoader(ds, batch_size=batch_size,
                    shuffle=True, num_workers=4)

    kp_dim = 33*4 + 468*3 + 21*3 + 21*3
    model = ISLModel(kp_dim, len(actions)).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr,
                       weight_decay=1e-4)
    crit  = nn.CrossEntropyLoss()

    for ep in range(1, epochs+1):
        model.train()
        total_loss = 0
        for imgs, kps, lbls in dl:
            out = model(imgs, kps)
            loss = crit(out, lbls.to(device))
            opt.zero_grad(); loss.backward(); opt.step()
            total_loss += loss.item()
        avg_loss = total_loss/len(dl)

        # training accuracy
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for imgs, kps, lbls in dl:
                preds = model(imgs, kps).argmax(dim=1)
                correct += (preds==lbls.to(device)).sum().item()
                total   += lbls.size(0)
        train_acc = correct/total

        print(f"Epoch {ep} — Loss: {avg_loss:.4f} — Train Acc: {train_acc*100:.2f}%")
        if train_acc >= early_stop_acc:
            torch.save(model.state_dict(), "hybrid_model_full.pth")
            print(f"Reached {train_acc*100:.2f}% ≥ {early_stop_acc*100:.0f}% — stopping.")
            break

    return model

# =============================================================================
# 10) RUNNING
# =============================================================================
if __name__=="__main__":
    VIDEO_DIR = "./dataset"
    KP_DIR    = "./extracted_keypoints"
    ACTIONS   = ["Death","Doctor","he","healthy","Hospital",
                 "I","Medicine","Patient","she","sick"]

    model = train_full(
        VIDEO_DIR, KP_DIR, ACTIONS,
        seq_len=60, batch_size=8,
        epochs=500, lr=1e-3, early_stop_acc=0.90
    )
    print("Done. Model saved to 'hybrid_model_full29.pth'")


Epoch 1 — Loss: 2.3664 — Train Acc: 15.22%


In [14]:
import os
import cv2
import torch
import numpy as np
from collections import deque
import mediapipe as mp
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn

# =============================================================================
# 1) DEVICE & TRANSFORM
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# =============================================================================
# 2) KEYPOINT EXTRACTION
# =============================================================================
mp_holistic = mp.solutions.holistic
def extract_all_keypoints(results):
    if results.pose_landmarks:
        pose = np.array([[l.x,l.y,l.z,l.visibility]
                         for l in results.pose_landmarks.landmark])
        a,b = pose[11,:2], pose[12,:2]
        center, scale = (a+b)/2, np.linalg.norm(a-b) or 1.0
        norm_pose = np.hstack([ (pose[:,:2]-center)/scale,
                                pose[:,2:3], pose[:,3:4] ])
    else:
        norm_pose = np.zeros((33,4))
    if results.face_landmarks:
        face = np.array([[l.x,l.y,l.z]
                         for l in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face[:,:2]-center)/scale,
                                face[:,2:3] ])
    else:
        norm_face = np.zeros((468,3))
    if results.left_hand_landmarks:
        lh = np.array([[l.x,l.y,l.z]
                       for l in results.left_hand_landmarks.landmark])
        w = lh[0,:2]
        norm_lh = np.hstack([ (lh[:,:2]-w)/scale, lh[:,2:3] ])
    else:
        norm_lh = np.zeros((21,3))
    if results.right_hand_landmarks:
        rh = np.array([[l.x,l.y,l.z]
                       for l in results.right_hand_landmarks.landmark])
        w = rh[0,:2]
        norm_rh = np.hstack([ (rh[:,:2]-w)/scale, rh[:,2:3] ])
    else:
        norm_rh = np.zeros((21,3))
    return np.concatenate([
        norm_pose.flatten(),
        norm_face.flatten(),
        norm_lh.flatten(),
        norm_rh.flatten()
    ])

# =============================================================================
# 3) FRAME SAMPLING
# =============================================================================
def sample_frames(video_path, N=60):
    if not os.path.isfile(video_path):
        raise FileNotFoundError(f"Video not found: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")
    frames = []
    while True:
        ret, frm = cap.read()
        if not ret: break
        frames.append(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    cap.release()
    if not frames:
        raise RuntimeError(f"No frames read: {video_path}")
    idxs = np.linspace(0, len(frames)-1, N).round().astype(int)
    return [frames[i] for i in idxs]

# =============================================================================
# 4) TEMPORAL CONVOLUTIONAL NETWORK
# =============================================================================
class TCNBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, d=1):
        super().__init__()
        pad = (k-1)*d//2
        self.conv1 = nn.Conv1d(in_ch, out_ch, k, padding=pad, dilation=d)
        self.conv2 = nn.Conv1d(out_ch, out_ch, k, padding=pad, dilation=d)
        self.down  = nn.Conv1d(in_ch, out_ch, 1) if in_ch!=out_ch else None
        self.relu  = nn.ReLU()
    def forward(self, x):
        y = self.relu(self.conv1(x))
        y = self.conv2(y)
        res = x if self.down is None else self.down(x)
        return self.relu(y + res)

class TemporalConvNet(nn.Module):
    def __init__(self, in_ch, levels=4, k=3, ch=512):
        super().__init__()
        layers = []
        for i in range(levels):
            layers.append(TCNBlock(in_ch if i==0 else ch, ch, k, 2**i))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        x = x.transpose(1,2)      # (B,C,T)
        y = self.net(x)
        return y.transpose(1,2)   # (B,T,C)

# =============================================================================
# 5) ATTENTION & MODEL
# =============================================================================
class Attention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(dim,1)
    def forward(self, x):
        w = torch.softmax(self.fc(x), dim=1)
        return (w * x).sum(dim=1)  # (B,dim)

class ISLModel(nn.Module):
    def __init__(self, kp_dim, n_classes):
        super().__init__()
        self.resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Identity()
        for p in self.resnet.parameters(): p.requires_grad=False
        self.img_proj  = nn.Linear(512,256)
        self.kp_proj   = nn.Linear(kp_dim,256)
        self.fuse_fc   = nn.Linear(512,512)
        self.tcn       = TemporalConvNet(512, levels=4)
        self.attn      = Attention(512)
        self.classifier= nn.Linear(512, n_classes)
        self.relu      = nn.ReLU()

    def forward(self, imgs, kps):
        B,T = imgs.size(0), imgs.size(1)
        img_feats = []
        for t in range(T):
            f = self.resnet(imgs[:,t].to(device))
            img_feats.append(self.relu(self.img_proj(f)))
        img_feats = torch.stack(img_feats, dim=1)       # (B,T,256)
        kp_feats  = self.relu(self.kp_proj(kps.to(device)))  # (B,T,256)
        x = torch.cat([img_feats, kp_feats], dim=-1)    # (B,T,512)
        x = self.relu(self.fuse_fc(x))                  # (B,T,512)
        x = self.tcn(x)                                 # (B,T,512)
        ctx = self.attn(x)                              # (B,512)
        return self.classifier(ctx)                     # (B,n_classes)

# =============================================================================
# 6) EVALUATION FUNCTIONS
# =============================================================================
def evaluate_once(video_path, model, actions, seq_len=60):
    print(f"[Single-Shot] {video_path}")
    frames = sample_frames(video_path, seq_len)
    holo = mp_holistic.Holistic(min_detection_confidence=0.5,
                                min_tracking_confidence=0.5)
    kps = [extract_all_keypoints(holo.process(cv2.cvtColor(img, cv2.COLOR_RGB2BGR)))
           for img in frames]
    holo.close()
    imgs_t = torch.stack([img_transform(img) for img in frames], dim=0) \
                .unsqueeze(0).to(device)
    kps_t  = torch.tensor(np.stack(kps), dtype=torch.float32) \
                .unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        probs = torch.softmax(model(imgs_t, kps_t), dim=1)[0].cpu().numpy()
    idx = int(probs.argmax())
    print(f"→ {actions[idx]} ({probs[idx]*100:.1f}%)")
    for i,a in enumerate(actions):
        print(f"   {a}: {probs[i]*100:.1f}%")

def sliding_window_prediction(video_path, model, actions,
                              window_size=60, thresh=0.7):
    print(f"[Sliding-Window] {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open: {video_path}")
    holo = mp_holistic.Holistic(min_detection_confidence=0.5,
                                min_tracking_confidence=0.5)
    win_f, win_k = deque(maxlen=window_size), deque(maxlen=window_size)
    while True:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        win_f.append(rgb)
        win_k.append(extract_all_keypoints(holo.process(rgb)))
        text = ""
        if len(win_k) == window_size:
            imgs_t = torch.stack([img_transform(f) for f in win_f], dim=0) \
                        .unsqueeze(0).to(device)
            kps_t  = torch.tensor(np.stack(win_k),
                                  dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                probs = torch.softmax(model(imgs_t, kps_t), dim=1)[0].cpu().numpy()
            idx = int(probs.argmax())
            if probs[idx] >= thresh:
                text = f"{actions[idx]} ({probs[idx]*100:.1f}%)"
        disp = frame.copy()
        if text:
            cv2.putText(disp, text, (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow("SLR Live", disp)
        if cv2.waitKey(1)&0xFF==ord('q'):
            break
    cap.release(); holo.close(); cv2.destroyAllWindows()

# =============================================================================
# 7) MAIN
# =============================================================================
if __name__ == "__main__":
    ACTIONS = ["Death","Doctor","he","healthy","Hospital",
               "I","Medicine","Patient","she","sick"]
    kp_dim = 33*4 + 468*3 + 21*3 + 21*3
    model  = ISLModel(kp_dim, len(ACTIONS)).to(device)
    model.load_state_dict(torch.load("hybrid_model_full.pth",
                                     map_location=device))

    # Single-shot evaluation:
    evaluate_once("videos/hospital_test.MOV", model, ACTIONS)

    # sliding‐window preview
    # sliding_window_prediction("videos/RiddhiDeath.mp4", model, ACTIONS)


[Single-Shot] videos/hospital_test.MOV


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

→ healthy (56.3%)
   Death: 0.0%
   Doctor: 0.0%
   he: 0.0%
   healthy: 56.3%
   Hospital: 0.0%
   I: 0.0%
   Medicine: 0.0%
   Patient: 0.0%
   she: 0.0%
   sick: 43.7%


In [2]:
# Modified Vannilla Bi-LSTM only training 
# 0.  LIBRARIES & DEVICE                                                      #
# ============================================================================#
import os, random, math, time, itertools
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
import mediapipe as mp

SEED   = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED);  np.random.seed(SEED);  random.seed(SEED)
print("Device:", DEVICE)

# ============================================================================#
# 1.  KEY‑POINT EXTRACTION & NORMALISATION                                    #
# ============================================================================#
mp_holistic = mp.solutions.holistic

def extract_all_keypoints(results):
    """Return 1‑D FP32 vector of pose+face+hands, shoulder‑normalised."""
    # pose
    if results.pose_landmarks:
        pose = np.array([[l.x, l.y, l.z, l.visibility]
                         for l in results.pose_landmarks.landmark])
        LS, RS = pose[11][:2], pose[12][:2]
        centre = (LS + RS) / 2
        scale  = np.linalg.norm(LS - RS) or 1.0
        norm_pose = np.hstack([(pose[:, :2]-centre)/scale, pose[:, 2:]])
    else:
        norm_pose = np.zeros((33, 4))

    # face
    if results.face_landmarks:
        face = np.array([[l.x, l.y, l.z]
                         for l in results.face_landmarks.landmark])
        norm_face = np.hstack([(face[:, :2]-centre)/scale, face[:, 2:]])
    else:
        norm_face = np.zeros((468, 3))

    # hands
    def _norm_hand(lms):
        wrist = lms[0, :2]
        return np.hstack([(lms[:, :2]-wrist)/scale, lms[:, 2:]])
    if results.left_hand_landmarks:
        lh = np.array([[l.x, l.y, l.z]
                       for l in results.left_hand_landmarks.landmark])
        norm_lh = _norm_hand(lh)
    else:
        norm_lh = np.zeros((21, 3))

    if results.right_hand_landmarks:
        rh = np.array([[l.x, l.y, l.z]
                       for l in results.right_hand_landmarks.landmark])
        norm_rh = _norm_hand(rh)
    else:
        norm_rh = np.zeros((21, 3))

    return np.concatenate([norm_pose.flatten(),
                           norm_face.flatten(),
                           norm_lh.flatten(),
                           norm_rh.flatten()]).astype(np.float32)

# ============================================================================#
# 2.  DATASET (reads *.npy sequences saved earlier)                           #
# ============================================================================#
class KeypointSequenceDataset(Dataset):
    """
    Folder layout:
        root/<class>/<video>/frame_*.npy   (exactly seq_len files)
    """
    def __init__(self, root, classes, seq_len=60,
                 augment=True, noise_std=0.01):
        self.samples, self.labels = [], []
        self.seq_len, self.augment, self.noise_std = seq_len, augment, noise_std
        for lbl, cls in enumerate(classes):
            cls_dir = os.path.join(root, cls)
            if not os.path.isdir(cls_dir):
                continue
            for vid in os.listdir(cls_dir):
                vdir = os.path.join(cls_dir, vid)
                if not os.path.isdir(vdir): continue
                frames = sorted([f for f in os.listdir(vdir) if f.endswith('.npy')],
                                key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frames) != seq_len: continue
                seq = [np.load(os.path.join(vdir, f)) for f in frames]
                self.samples.append(np.stack(seq).astype(np.float32))
                self.labels .append(lbl)
        self.samples, self.labels = np.asarray(self.samples), np.asarray(self.labels)

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        seq = self.samples[idx].copy()
        if self.augment:
            seq += np.random.normal(0, self.noise_std, seq.shape).astype(np.float32)
        return torch.from_numpy(seq), torch.tensor(self.labels[idx])

# ============================================================================#
# 3.  BI‑LSTM + ATTENTION MODEL                                               #
# ============================================================================#
class SignRecognizer(nn.Module):
    def __init__(self, in_dim, n_classes, hidden=256,
                 n_layers=2, drop=0.3):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden, n_layers,
                            batch_first=True, bidirectional=True,
                            dropout=drop)
        self.attn = nn.Linear(hidden*2, 1)
        self.norm = nn.LayerNorm(hidden*2)
        self.head = nn.Sequential(
            nn.Linear(hidden*2, hidden),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden, n_classes)
        )
    def forward(self, x):            # (B,T,D)
        h, _   = self.lstm(x)        # (B,T,2H)
        α      = torch.softmax(self.attn(h).squeeze(-1), 1)   # (B,T)
        ctx    = (h * α.unsqueeze(-1)).sum(1)                 # (B,2H)
        ctx    = self.norm(ctx)
        return self.head(ctx)

# ============================================================================#
# 4.  TRAIN / VALIDATION SPLIT & TRAINER                                      #
# ============================================================================#
def stratified_split(ds, test_ratio=0.2):
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio,
                                      random_state=SEED)
    idx_train, idx_val = next(splitter.split(np.zeros(len(ds)), ds.labels))
    return Subset(ds, idx_train), Subset(ds, idx_val)

def train_model(root, classes,
                seq_len      = 60,
                batch        = 32,
                lr           = 5e-4,     # ← lower LR
                epochs       = 3000,
                early_patience = 25,     # ← more patience
                min_delta    = 1e-4):    # ← compare loss, not acc
    # --------------------------------------------------------------------- #
    #  DATA
    # --------------------------------------------------------------------- #
    full_ds   = KeypointSequenceDataset(root, classes, seq_len, augment=True)
    train_ds, val_ds = stratified_split(full_ds, 0.20)
    train_dl = DataLoader(train_ds, batch, shuffle=True,  drop_last=True)
    val_dl   = DataLoader(val_ds  , batch, shuffle=False, drop_last=False)

    # --------------------------------------------------------------------- #
    #  MODEL, OPTIMISER, SCHEDULER
    # --------------------------------------------------------------------- #
    model = SignRecognizer(full_ds.samples.shape[2],
                           len(classes)).to(DEVICE)
    opt   = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

    # cosine schedule with 10‑epoch warm‑up
    def lr_lambda(ep, warm=10):
        if ep < warm:
            return (ep + 1) / warm
        return 0.5 * (1 + math.cos(math.pi * (ep - warm) / max(1, epochs - warm)))
    sched = optim.lr_scheduler.LambdaLR(opt, lr_lambda)

    best_loss, bad_epochs = float('inf'), 0
    crit = nn.CrossEntropyLoss()

    # --------------------------------------------------------------------- #
    #  TRAIN LOOP
    # --------------------------------------------------------------------- #
    for ep in range(1, epochs + 1):

        # Turn off Gaussian‑noise augmentation halfway through training
        if ep == epochs // 2:
            train_ds.dataset.augment = False

        # ---------------------- TRAIN ---------------------- #
        model.train()
        tr_loss = tr_correct = 0
        for X, y in train_dl:
            X, y = X.to(DEVICE), y.to(DEVICE)
            opt.zero_grad()
            out = model(X)
            loss = crit(out, y)
            loss.backward()
            opt.step()
            tr_loss   += loss.item()
            tr_correct += (out.argmax(1) == y).sum().item()
        tr_acc = 100 * tr_correct / len(train_ds)

        # ---------------------- VALID ---------------------- #
        model.eval()
        val_loss = val_correct = 0
        with torch.no_grad():
            for X, y in val_dl:
                X, y = X.to(DEVICE), y.to(DEVICE)
                out  = model(X)
                val_loss   += crit(out, y).item()
                val_correct += (out.argmax(1) == y).sum().item()
        val_acc  = 100 * val_correct / len(val_ds)

        sched.step()  # update LR

        print(f"Ep {ep:04d}  "
              f"train {tr_acc:6.2f}%  val {val_acc:6.2f}%  "
              f"val‑loss {val_loss/len(val_dl):.4f}  "
              f"lr {sched.get_last_lr()[0]:.5f}")

        # ---------------- EARLY‑STOPPING ------------------- #
        if val_loss < best_loss - min_delta:
            best_loss, bad_epochs = val_loss, 0
            torch.save(model.state_dict(), "sign_recognizer_best.pth")
        else:
            bad_epochs += 1
            if bad_epochs >= early_patience:
                print("Early‑stop triggered (no val‑loss improvement).")
                break

    print(f"Best val loss: {best_loss:.4f}")
    model.load_state_dict(torch.load("sign_recognizer_best.pth"))
    model.eval()
    return model

# ============================================================================#
# 5.  OPTIONAL EVALUATION (CONFUSION MATRIX)                                  #
# ============================================================================#
def evaluate(model, root, classes, seq_len=60, batch=32):
    ds = KeypointSequenceDataset(root, classes, seq_len, augment=False)
    dl = DataLoader(ds, batch)
    preds, trues = [], []
    with torch.no_grad():
        for X,y in dl:
            p = model(X.to(DEVICE)).argmax(1).cpu().numpy()
            preds.extend(p); trues.extend(y.numpy())
    print("Accuracy %.2f%%" % (100 * (np.array(preds)==np.array(trues)).mean()))
    print(confusion_matrix(trues, preds))
    print(classification_report(trues, preds, target_names=classes, digits=3))

# ============================================================================#
# 6.  SEGMENTER + TRANSLATOR                                                  #
# ============================================================================#
def segment_video(path, idle_win=8, vel_thr=0.02, min_len=20, max_len=90):
    segs, buf, idle = [], [], 0
    prev_w = None
    with mp_holistic.Holistic(min_detection_confidence=0.4,
                              min_tracking_confidence=0.4) as holo:
        cap = cv2.VideoCapture(path)
        while True:
            ok, frame = cap.read()
            if not ok: break
            res = holo.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            kp  = extract_all_keypoints(res)
            # wrist velocity
            w = (np.array([res.right_hand_landmarks.landmark[0].x,
                           res.right_hand_landmarks.landmark[0].y])
                 if res.right_hand_landmarks else
                 (np.array([res.left_hand_landmarks.landmark[0].x,
                            res.left_hand_landmarks.landmark[0].y])
                  if res.left_hand_landmarks else prev_w))
            vel = np.linalg.norm(w - prev_w) if prev_w is not None else 0
            prev_w = w
            buf.append(kp)
            idle = idle+1 if vel<vel_thr else 0
            if idle>=idle_win and len(buf)>=min_len:
                seg = np.stack(buf[:-idle_win])
                if len(seg)>=min_len: segs.append(seg[:max_len])
                buf, idle = [], 0
        if len(buf)>=min_len:
            segs.append(np.stack(buf[:max_len]))
        cap.release()
    return segs

def pad_or_trim(seq, target=60):
    if len(seq)==target: return seq.astype(np.float32)
    if len(seq)>target:
        idx = np.linspace(0,len(seq)-1,target).round().astype(int)
        return seq[idx].astype(np.float32)
    pad = np.repeat(seq[-1][None,:], target-len(seq),0)
    return np.concatenate([seq,pad],0).astype(np.float32)

def translate_video(path, model, classes,
                    seq_len=60, conf=0.6):
    words=[]
    for seg in segment_video(path):
        seq = pad_or_trim(seg, seq_len)
        with torch.no_grad():
            probs = torch.softmax(
                model(torch.from_numpy(seq).unsqueeze(0).to(DEVICE)),1
            ).cpu().numpy()[0]
        best = probs.argmax()
        if probs[best] >= conf:
            words.append(classes[best])
    return words

# ============================================================================#
# 7.  --- EXAMPLE USAGE ------------------------------------------------------ #
# ============================================================================#
classes = [  # replace with your full 35‑word list in ***identical folder order***
    "Death","Doctor","he","healthy","Hospital",
    "I","Medicine","Patient","she","sick"
]
root_dir = "./extracted_keypoints"     # <‑‑ your *.npy folders
continuous_clip = "./videos/SickSourikSampleVideo - Trim2.mp4"  # <‑‑ test clip

if not os.path.isfile("sign_recognizer_best.pth"):
    model = train_model(root_dir, classes)
else:
    # quick load without retraining
    dummy = KeypointSequenceDataset(root_dir, classes, augment=False)
    model = SignRecognizer(dummy.samples.shape[2], len(classes)).to(DEVICE)
    model.load_state_dict(torch.load("sign_recognizer_best.pth")); model.eval()

# optional sanity‑check on all isolated videos
evaluate(model, root_dir, classes)

# run translation on a continuous video
print("Translation:",
      " ".join(translate_video(continuous_clip, model, classes)))


Device: cuda
Accuracy 96.19%
[[29  0  0  0  0  0  0  0  0  0]
 [ 0 29  0  0  0  0  0  0  0  0]
 [ 0  0 28  0  0  0  0  0  1  0]
 [ 0  0  0 29  0  0  0  0  0  0]
 [ 0  0  0  0 29  0  0  0  0  0]
 [ 0  0  0  0  0 29  0  0  0  0]
 [ 1  0  0  0  0  0 28  0  0  0]
 [ 0  0  0  0  0  0  0 28  0  0]
 [ 0  0  9  0  0  0  0  0 20  0]
 [ 0  0  0  0  0  0  0  0  0 29]]
              precision    recall  f1-score   support

       Death      0.967     1.000     0.983        29
      Doctor      1.000     1.000     1.000        29
          he      0.757     0.966     0.848        29
     healthy      1.000     1.000     1.000        29
    Hospital      1.000     1.000     1.000        29
           I      1.000     1.000     1.000        29
    Medicine      1.000     0.966     0.982        29
     Patient      1.000     1.000     1.000        28
         she      0.952     0.690     0.800        29
        sick      1.000     1.000     1.000        29

    accuracy                          0.962 

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Translation: 


In [1]:
#  NEW BiLSTM!!! Single‑shot inference on ONE isolated‑sign video clip
#  ------------------------------------------------------------
#  1.  Put the trained weights file (sign_recognizer_best.pth)
#      in the same folder as this notebook – or change WEIGHTS_PATH.
#  2.  Fill in your full 'classes' list (exact order used during
#      training) and the path to the test video.
#  3.  Run the cell – it prints the predicted class + probabilities.
# ============================================================

import os, cv2, math, numpy as np, torch, torch.nn as nn
import mediapipe as mp

# ------------------------------------------------------------------
#  CONFIGURATION
# ------------------------------------------------------------------
WEIGHTS_PATH = "sign_recognizer_best.pth"          # ← trained weights
TEST_VIDEO   = "./videos/Medicine.MP4"         # ← path to your video
SEQ_LEN      = 60                                  # frames per example
CLASSES      = [                                   # ← update to ALL your classes
    "Death", "Doctor", "he", "healthy", "Hospital",
    "I", "Medicine", "Patient", "she", "sick"
]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------------------------------------------
#  MEDIA‑PIPE KEYPOINT EXTRACTION
# ------------------------------------------------------------------
mp_holistic = mp.solutions.holistic

def extract_all_keypoints(results):
    """Return a (1662,) FP32 array – pose(33*4)+face(468*3)+hands(2×21*3)"""
    # ---------- pose ----------
    if results.pose_landmarks:
        pose = np.array([[l.x, l.y, l.z, l.visibility] for l in results.pose_landmarks.landmark])
        LS, RS = pose[11][:2], pose[12][:2]                  # shoulders
        centre = (LS + RS) / 2
        scale  = np.linalg.norm(LS - RS) or 1.0
        pose   = np.hstack([(pose[:, :2] - centre) / scale, pose[:, 2:]])
    else:
        pose = np.zeros((33, 4))

    # ---------- face ----------
    if results.face_landmarks:
        face = np.array([[l.x, l.y, l.z] for l in results.face_landmarks.landmark])
        face = np.hstack([(face[:, :2] - centre) / scale, face[:, 2:]])
    else:
        face = np.zeros((468, 3))

    # ---------- hands ----------
    def _norm_hand(lms):
        wrist = lms[0, :2]
        return np.hstack([(lms[:, :2] - wrist) / scale, lms[:, 2:]])

    if results.left_hand_landmarks:
        lh = np.array([[l.x, l.y, l.z] for l in results.left_hand_landmarks.landmark])
        lh = _norm_hand(lh)
    else:
        lh = np.zeros((21, 3))

    if results.right_hand_landmarks:
        rh = np.array([[l.x, l.y, l.z] for l in results.right_hand_landmarks.landmark])
        rh = _norm_hand(rh)
    else:
        rh = np.zeros((21, 3))

    return np.concatenate([pose.flatten(), face.flatten(),
                           lh.flatten(), rh.flatten()]).astype(np.float32)

def sample_frames_uniformly(frames, k=60):
    """Return exactly k indices spread uniformly over len(frames)."""
    if len(frames) == 0:
        return []
    if len(frames) == k:
        return frames
    idx = np.linspace(0, len(frames) - 1, k).round().astype(int)
    return [frames[i] for i in idx]

# ------------------------------------------------------------------
#  MODEL (must match architecture used during training)
# ------------------------------------------------------------------
class SignRecognizer(nn.Module):
    def __init__(self, in_dim=1662, n_classes=10, hidden=256,
                 n_layers=2, drop=0.3):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden, n_layers,
                            batch_first=True, bidirectional=True,
                            dropout=drop)
        self.attn = nn.Linear(hidden * 2, 1)
        self.norm = nn.LayerNorm(hidden * 2)
        self.head = nn.Sequential(
            nn.Linear(hidden * 2, hidden),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden, n_classes)
        )

    def forward(self, x):                     # x: (B,T,D)
        h, _ = self.lstm(x)                   # (B,T,2H)
        α    = torch.softmax(self.attn(h).squeeze(-1), dim=1)  # (B,T)
        ctx  = (h * α.unsqueeze(-1)).sum(1)   # (B,2H)
        ctx  = self.norm(ctx)
        return self.head(ctx)                 # (B,C)

# ------------------------------------------------------------------
#  LOAD MODEL WEIGHTS
# ------------------------------------------------------------------
model = SignRecognizer(in_dim=1662, n_classes=len(CLASSES)).to(DEVICE)
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=DEVICE))
model.eval()
print("Loaded weights from", WEIGHTS_PATH)

# ------------------------------------------------------------------
#  INFERENCE FUNCTION
# ------------------------------------------------------------------
def infer_single_video(video_path, model, classes, seq_len=60):
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        all_frames.append(frame)
    cap.release()

    if len(all_frames) == 0:
        raise RuntimeError("No frames read from video!")

    frames = sample_frames_uniformly(all_frames, k=seq_len)

    kp_seq = []
    with mp_holistic.Holistic(
            static_image_mode=False,
            model_complexity=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as holistic:
        for f in frames:
            img_rgb = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
            res = holistic.process(img_rgb)
            kp_seq.append(extract_all_keypoints(res))

    seq = np.stack(kp_seq).astype(np.float32)        # (T,D)
    tensor = torch.from_numpy(seq).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logits = model(tensor)
        probs  = torch.softmax(logits, dim=1).cpu().numpy()[0]
        best   = probs.argmax()

    return classes[best], probs

# ------------------------------------------------------------------
#  RUN INFERENCE
# ------------------------------------------------------------------
pred, prob_vector = infer_single_video(TEST_VIDEO, model, CLASSES, SEQ_LEN)

print(f"\nPredicted class :  {pred}")
print("Probabilities   :")
for cls, p in zip(CLASSES, prob_vector):
    print(f"  {cls:<10s}: {p:.4f}")


Loaded weights from sign_recognizer_best.pth


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar


Predicted class :  sick
Probabilities   :
  Death     : 0.0026
  Doctor    : 0.0053
  he        : 0.0016
  healthy   : 0.0013
  Hospital  : 0.0002
  I         : 0.3817
  Medicine  : 0.0001
  Patient   : 0.0019
  she       : 0.0014
  sick      : 0.6040


In [6]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from collections import deque
import mediapipe as mp

# ------------------------------------------------------------------
#  CONFIG
# ------------------------------------------------------------------
WEIGHTS_PATH = "sign_recognizer_best.pth"
TEST_VIDEO   = "./videos/RiddhiDeath.mp4"
SEQ_LEN      = 60
SMOOTH_W     = 10
THRESH       = 0.7
CLASSES      = [
    "Death","Doctor","he","healthy","Hospital",
    "I","Medicine","Patient","she","sick"
]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------------------------------------------
#  MEDIA-PIPE KEYPOINT EXTRACTION
# ------------------------------------------------------------------
mp_holistic = mp.solutions.holistic

def extract_all_keypoints(results):
    if results.pose_landmarks:
        pose = np.array([[l.x,l.y,l.z,l.visibility]
                         for l in results.pose_landmarks.landmark])
        LS,RS = pose[11,:2], pose[12,:2]
        centre = (LS+RS)/2
        scale  = np.linalg.norm(LS-RS) or 1.0
        pose = np.hstack([ (pose[:,:2]-centre)/scale, pose[:,2:] ])
    else:
        pose = np.zeros((33,4),dtype=np.float32)
    if results.face_landmarks:
        face = np.array([[l.x,l.y,l.z]
                         for l in results.face_landmarks.landmark])
        face = np.hstack([ (face[:,:2]-centre)/scale, face[:,2:] ])
    else:
        face = np.zeros((468,3),dtype=np.float32)
    def _norm_hand(lms):
        wrist = lms[0,:2]
        return np.hstack([ (lms[:,:2]-wrist)/scale, lms[:,2:] ])
    if results.left_hand_landmarks:
        lh = np.array([[l.x,l.y,l.z] for l in results.left_hand_landmarks.landmark])
        lh = _norm_hand(lh)
    else:
        lh = np.zeros((21,3),dtype=np.float32)
    if results.right_hand_landmarks:
        rh = np.array([[l.x,l.y,l.z] for l in results.right_hand_landmarks.landmark])
        rh = _norm_hand(rh)
    else:
        rh = np.zeros((21,3),dtype=np.float32)
    return np.concatenate([pose.flatten(),
                           face.flatten(),
                           lh.flatten(),
                           rh.flatten()]).astype(np.float32)

# ------------------------------------------------------------------
#  FRAME SAMPLING UTILITY
# ------------------------------------------------------------------
def sample_frames_uniformly(frames, k=SEQ_LEN):
    if len(frames)==0: return []
    if len(frames)==k: return frames
    idxs = np.linspace(0, len(frames)-1, k).round().astype(int)
    return [frames[i] for i in idxs]

# ------------------------------------------------------------------
#  MODEL DEFINITION (must match your training)
# ------------------------------------------------------------------
class SignRecognizer(nn.Module):
    def __init__(self, in_dim=1662, n_classes=10, hidden=256,
                 n_layers=2, drop=0.3):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden, n_layers,
                            batch_first=True, bidirectional=True,
                            dropout=drop)
        self.attn = nn.Linear(hidden*2,1)
        self.norm = nn.LayerNorm(hidden*2)
        self.head = nn.Sequential(
            nn.Linear(hidden*2, hidden),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden, n_classes)
        )
    def forward(self,x):
        h,_ = self.lstm(x)                         # (B,T,2H)
        α   = torch.softmax(self.attn(h).squeeze(-1), dim=1)
        ctx = (h * α.unsqueeze(-1)).sum(1)         # (B,2H)
        ctx = self.norm(ctx)
        return self.head(ctx)                     # (B,C)

# ------------------------------------------------------------------
#  LOAD MODEL
# ------------------------------------------------------------------
model = SignRecognizer(in_dim=1662, n_classes=len(CLASSES))\
        .to(DEVICE)
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=DEVICE))
model.eval()
print("Loaded weights from", WEIGHTS_PATH)

# ------------------------------------------------------------------
#  PROBABILITY VISUALISATION
# ------------------------------------------------------------------
def prob_viz(probs, classes, frame, max_bar=200, start_y=60, line_h=30):
    out = frame.copy()
    for i,p in enumerate(probs):
        w = int(p*max_bar)
        y0 = start_y + i*line_h
        cv2.rectangle(out, (0,y0),(w,y0+line_h-5),(50,200,50),-1)
        cv2.putText(out,
                    f"{classes[i]}:{p:.2f}",
                    (5,y0+line_h-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                    (255,255,255),1,cv2.LINE_AA)
    return out

# ------------------------------------------------------------------
#  SLIDING-WINDOW INFERENCE
# ------------------------------------------------------------------
def sliding_window(video_path):
    cap = cv2.VideoCapture(video_path)
    holo = mp_holistic.Holistic(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)
    Kwin = deque(maxlen=SEQ_LEN)
    preds= deque(maxlen=SMOOTH_W)
    sentence=[]
    cv2.namedWindow("ISL Live", cv2.WINDOW_NORMAL)

    while True:
        ok, frame = cap.read()
        if not ok:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        # extract keypoints
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = holo.process(rgb)
        kp = extract_all_keypoints(res)
        Kwin.append(kp)

        # when full, run model
        if len(Kwin)==SEQ_LEN:
            seq = np.stack(Kwin).astype(np.float32)      # (T,D)
            x   = torch.from_numpy(seq)\
                       .unsqueeze(0).to(DEVICE)          # (1,T,D)
            with torch.no_grad():
                out   = model(x)                         # (1,C)
                prob  = torch.softmax(out,1)[0].cpu().numpy()
                pidx  = int(prob.argmax())
            preds.append(pidx)

            # temporal smoothing
            if len(preds)==SMOOTH_W and len(set(preds))==1 and prob[pidx]>=THRESH:
                w = CLASSES[pidx]
                if not sentence or sentence[-1]!=w:
                    sentence.append(w)

            # overlay probabilities
            vis = prob_viz(prob, CLASSES, frame)
        else:
            vis = frame

        # overlay text
        cv2.putText(vis," ".join(sentence),(10,40),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        cv2.imshow("ISL Live", vis)

        if cv2.waitKey(1)&0xFF==ord('q'):
            break

    cap.release()
    holo.close()
    cv2.destroyAllWindows()

# ------------------------------------------------------------------
#  RUN IT
# ------------------------------------------------------------------
if __name__=="__main__":
    sliding_window(TEST_VIDEO)


Loaded weights from sign_recognizer_best.pth


libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

In [ ]:
#!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [5]:
#!pip install accelerate>=0.26.0

In [ ]:
#Blocking LLM + Glosses Detection via LSTM

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from transformers import pipeline

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        norm_pose = np.hstack([ (pose_points[:,0:2] - center) / scale, 
                                pose_points[:,2:3], 
                                pose_points[:,3:4] ])
    else:
        norm_pose = np.zeros((33, 4))
    
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face_points[:,0:2] - center) / scale, 
                                face_points[:,2:3] ])
    else:
        norm_face = np.zeros((468, 3))
    
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]
        norm_left_hand = np.hstack([ (left_hand_points[:,0:2] - wrist) / scale, 
                                     left_hand_points[:,2:3] ])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([ (right_hand_points[:,0:2] - wrist) / scale, 
                                      right_hand_points[:,2:3] ])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS WITH AUGMENTATION OPTION
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    def __init__(self, data_dir, actions, sequence_length=60, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            for video_folder in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_folder)
                if not os.path.isdir(video_path):
                    continue
                frame_files = [f for f in os.listdir(video_folder) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    frames = []
                    for f in frame_files:
                        fpath = os.path.join(video_folder, f)
                        frames.append(np.load(fpath))
                    sequence = np.stack(frames, axis=0)
                    self.data.append(sequence)
                    self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if self.augment:
            sequence = augment_sequence(sequence)
        sample = torch.tensor(sequence, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    sample, _ = dataset[0]
    input_size = sample.shape[1]
    num_classes = len(actions)
    
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        elapsed = time.time() - start_time
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    
    torch.save(model.state_dict(), "action_detection_model60.pth")
    print("Model weights saved as 'action_detection_model60.pth'")
    return model

# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# 9. LLM INTEGRATION: GLOSS-TO-SENTENCE TRANSLATION USING LLAMA-3.2-1B
# =============================================================================
# Ensure your transformers version is updated: pip install --upgrade transformers
model_id = "meta-llama/Llama-3.2-3B"
HF_TOKEN = os.environ.get("HF_TOKEN")
llm_generator = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=HF_TOKEN,
    trust_remote_code=True
)

prompt_template = (
    "You are a professional sign language interpreter. "
    "Convert the following sign language glosses into a clear, concise, and natural English sentence. "
    "Do not include any parts of this instruction in your response; output only the final sentence. "
    "Do NOT make a complicated sentence, if I provide you ['I','Sick'], you should output 'I am Sick.'"
    "Glosses: {}"
)

def gloss_to_sentence(glosses, generator, prompt_template=prompt_template):
    # Only generate a sentence if we have at least two distinct glosses.
    if len(set(glosses)) < 2:
        return ""
    gloss_string = " ".join(glosses)
    prompt = prompt_template.format(gloss_string)
    result = generator(prompt, max_new_tokens=30, do_sample=True, temperature=0.7)
    sentence = result[0]['generated_text']
    # Remove the prompt from the output if present.
    if prompt in sentence:
        sentence = sentence.replace(prompt, "").strip()
    return sentence


# =============================================================================
# 10. REAL-TIME SLIDING WINDOW PREDICTION WITH CONFIDENCE THRESHOLDING AND LLM INTEGRATION
# =============================================================================
def prob_viz(probabilities, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(probabilities):
        width = int(prob * 200)
        top_left = (0, 60 + num * 40)
        bottom_right = (width, 90 + num * 40)
        cv2.rectangle(output_frame, top_left, bottom_right, colors[num], -1)
        cv2.putText(output_frame, f"{actions[num]}: {prob:.2f}", (10, 85 + num * 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

def sliding_window_prediction(video_file, model, actions, window_size=60, smoothing_window=20, threshold=0.7, gap_frames=20):
    colors = [(245,117,16), (117,245,16), (16,117,245), (0,255,0)]
    sequence = []  # Current sliding window of keypoints
    predictions = []  # Recent predicted class indices
    prediction_confidences = []  # Corresponding confidence scores
    displayed_prediction = "NA"
    accumulated_glosses = []  # Accumulated distinct glosses for the current sentence
    gap_counter = 0  # Count consecutive frames with "NA"
    sentence_generated = ""
    
    cap = cv2.VideoCapture(video_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            keypoints = extract_all_keypoints(results)
            sequence.append(keypoints)
            if len(sequence) > window_size:
                sequence = sequence[-window_size:]
            
            if len(sequence) == window_size:
                seq_array = np.stack(sequence, axis=0)
                seq_tensor = torch.tensor(seq_array, dtype=torch.float32).unsqueeze(0).to(device)
                
                model.eval()
                with torch.no_grad():
                    outputs = model(seq_tensor)
                    prob = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                    pred_idx = np.argmax(prob)
                    conf = prob[pred_idx]
                    predictions.append(pred_idx)
                    prediction_confidences.append(conf)
                
                if len(prediction_confidences) >= smoothing_window:
                    recent_preds = predictions[-smoothing_window:]
                    recent_confs = prediction_confidences[-smoothing_window:]
                    if (len(set(recent_preds)) == 1) and all(c >= threshold for c in recent_confs):
                        current_prediction = actions[pred_idx]
                    else:
                        current_prediction = "NA"
                    
                    if current_prediction != displayed_prediction:
                        displayed_prediction = current_prediction
                        
                    # Only add if the accumulated glosses is empty or last gloss is different.
                    if displayed_prediction != "NA":
                        if not accumulated_glosses or accumulated_glosses[-1] != displayed_prediction:
                            accumulated_glosses.append(displayed_prediction)
                        gap_counter = 0
                    else:
                        gap_counter += 1
                        
                    # Only trigger LLM generation if at least 2 distinct glosses are accumulated.
                    if gap_counter >= gap_frames and len(accumulated_glosses) >= 2:
                        print("Glosses sent to LLM for parsing : ",accumulated_glosses)
                        sentence_generated = gloss_to_sentence(accumulated_glosses, llm_generator)
                        print("LLM Generated Sentence:", sentence_generated)
                        accumulated_glosses = []
                        gap_counter = 0

                print("Current glosses : ",accumulated_glosses)
                vis_frame = prob_viz(prob, actions, image, colors)
                cv2.rectangle(vis_frame, (0, 0), (640, 40), (245,117,16), -1)
                cv2.putText(vis_frame, displayed_prediction, (3, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                if sentence_generated:
                    cv2.putText(vis_frame, sentence_generated, (3, 70),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Sliding Window Action Detection", vis_frame)
            else:
                cv2.imshow("Sliding Window Action Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# =============================================================================
# 11. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    actions = actions = [
    "Baby", "Bill", "Boy", "Brother", "Daughter", "Death",
    "Doctor", "Family", "Father", "Girl", "he", "healthy",
    "Hospital", "Husband", "I", "it", "Man", "Medicine",
    "Money", "Mother", "Parent", "Patient", "Price",
    "she", "sick", "Sign", "Sister", "Son", "Student",
    "they", "we", "Wife", "Woman", "you", "you (plural)"
    ]
    extracted_keypoints_dir = "./extracted_keypoints"
    raw_dataset_dir = "./dataset"
    
    # Uncomment the block below to extract keypoints from raw training videos.
    # for action in actions:
    #     action_raw_folder = os.path.join(raw_dataset_dir, action)
    #     action_extracted_folder = os.path.join(extracted_keypoints_dir, action)
    #     os.makedirs(action_extracted_folder, exist_ok=True)
    #     video_files = [vf for vf in os.listdir(action_raw_folder) if vf.lower().endswith((".mp4", ".avi", ".mov"))]
    #     for video_file in video_files:
    #         video_path = os.path.join(action_raw_folder, video_file)
    #         video_basename = os.path.splitext(video_file)[0]
    #         video_output_folder = os.path.join(action_extracted_folder, video_basename)
    #         os.makedirs(video_output_folder, exist_ok=True)
    #         print(f"Processing video: {video_path}")
    #         process_video_and_save_frames(video_path, video_output_folder, target_frames=60)
    
    input_size = 1662
    num_classes = len(actions)
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load("action_detection_model60.pth", map_location=device))
    
    test_video_file = "./videos/SourikSampleVideo.mp4"  # Adjust path as needed
    sliding_window_prediction(test_video_file, model, actions, window_size=60, smoothing_window=20, threshold=0.7)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix

Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current glosses :  ['I']
Current

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Current glosses :  ['I', 'Sick']
Current glosses :  ['I', 'Sick']
Current glosses :  ['I', 'Sick']
Current glosses :  ['I', 'Sick']
Current glosses :  ['I', 'Sick']
Current glosses :  ['I', 'Sick']
Glosses sent to LLM for parsing :  ['I', 'Sick']
LLM Generated Sentence: You Are SickWe SickYou SickYou SickI SickI SickYou SickYou SickYou SickYou SickYou SickYou SickYou SickYou SickYou
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  []
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  ['Sick']
Current glosses :  [

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Glosses sent to LLM for parsing :  ['Sick', 'Price']
LLM Generated Sentence: , Sick, I, You, Me, Him, Her, Us, Them, It, My, Your, His, Her, Our, Their
Current glosses :  []


In [ ]:
#Async LLM + Glosses Detection via LSTM

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mediapipe as mp
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from transformers import pipeline
import concurrent.futures

# =============================================================================
# 1. DEVICE CONFIGURATION
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 2. KEYPOINT EXTRACTION AND NORMALIZATION
# =============================================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_all_keypoints(results):
    if results.pose_landmarks:
        pose_points = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])
        try:
            left_shoulder = pose_points[11][:2]
            right_shoulder = pose_points[12][:2]
            center = (left_shoulder + right_shoulder) / 2.0
            scale = np.linalg.norm(left_shoulder - right_shoulder)
            if scale < 1e-6:
                scale = 1.0
        except Exception as e:
            center = np.array([0.5, 0.5])
            scale = 1.0
        norm_pose = np.hstack([ (pose_points[:,0:2] - center) / scale,
                                pose_points[:,2:3],
                                pose_points[:,3:4] ])
    else:
        norm_pose = np.zeros((33, 4))
    
    if results.face_landmarks:
        face_points = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark])
        norm_face = np.hstack([ (face_points[:,0:2] - center) / scale,
                                face_points[:,2:3] ])
    else:
        norm_face = np.zeros((468, 3))
    
    if results.left_hand_landmarks:
        left_hand_points = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark])
        wrist = left_hand_points[0][:2]
        norm_left_hand = np.hstack([ (left_hand_points[:,0:2] - wrist) / scale,
                                     left_hand_points[:,2:3] ])
    else:
        norm_left_hand = np.zeros((21, 3))
    
    if results.right_hand_landmarks:
        right_hand_points = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
        wrist = right_hand_points[0][:2]
        norm_right_hand = np.hstack([ (right_hand_points[:,0:2] - wrist) / scale,
                                      right_hand_points[:,2:3] ])
    else:
        norm_right_hand = np.zeros((21, 3))
    
    return np.concatenate([norm_pose.flatten(), norm_face.flatten(), norm_left_hand.flatten(), norm_right_hand.flatten()])

# =============================================================================
# 3. DATA AUGMENTATION FUNCTION FOR KEYPOINT SEQUENCES
# =============================================================================
def augment_sequence(sequence, noise_std=0.01):
    noise = np.random.normal(0, noise_std, sequence.shape)
    return sequence + noise

# =============================================================================
# 4. FRAME SAMPLING AND VIDEO PROCESSING
# =============================================================================
def sample_frames_uniformly(frames, target_count=60):
    num_frames = len(frames)
    if num_frames == 0:
        return []
    if num_frames == target_count:
        return frames
    sampled = []
    if num_frames > target_count:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    else:
        for i in range(target_count):
            idx = int(round(i * (num_frames - 1) / (target_count - 1)))
            sampled.append(frames[idx])
    return sampled

def process_video_and_save_frames(video_path, output_folder, target_frames=60):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    sampled_frames = sample_frames_uniformly(all_frames, target_count=target_frames)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        for i, frm in enumerate(sampled_frames):
            image = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            keypoints = extract_all_keypoints(results)
            output_file = os.path.join(output_folder, f"frame_{i}.npy")
            np.save(output_file, keypoints)
            print(f"Saved {output_file}, shape: {keypoints.shape}")

# =============================================================================
# 5. DATASET CLASS WITH AUGMENTATION OPTION
# =============================================================================
class SignLanguageVideoDataset(Dataset):
    def __init__(self, data_dir, actions, sequence_length=60, augment=False):
        self.data = []
        self.labels = []
        self.actions = actions
        self.sequence_length = sequence_length
        self.augment = augment
        
        for label, action in enumerate(actions):
            action_folder = os.path.join(data_dir, action)
            if not os.path.isdir(action_folder):
                continue
            for video_folder in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_folder)
                if not os.path.isdir(video_path):
                    continue
                frame_files = [f for f in os.listdir(video_folder) if f.endswith('.npy')]
                frame_files = sorted(frame_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
                if len(frame_files) == sequence_length:
                    frames = []
                    for f in frame_files:
                        fpath = os.path.join(video_folder, f)
                        frames.append(np.load(fpath))
                    sequence = np.stack(frames, axis=0)
                    self.data.append(sequence)
                    self.labels.append(label)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if self.augment:
            sequence = augment_sequence(sequence)
        sample = torch.tensor(sequence, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

# =============================================================================
# 6. MODEL DEFINITION WITH DROPOUT REGULARIZATION
# =============================================================================
class ActionModelPyTorch(nn.Module):
    def __init__(self, input_size, num_classes, dropout_prob=0.3):
        super(ActionModelPyTorch, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=64, batch_first=True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(256, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = out[:, -1, :]
        
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc2(out)
        return out

# =============================================================================
# 7. TRAINING FUNCTION
# =============================================================================
def train_model(data_dir, actions, sequence_length=60, batch_size=32,
                num_epochs=250, learning_rate=0.001, augment=False):
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=augment)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    sample, _ = dataset[0]
    input_size = sample.shape[1]
    num_classes = len(actions)
    
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        elapsed = time.time() - start_time
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    
    torch.save(model.state_dict(), "action_detection_model60.pth")
    print("Model weights saved as 'action_detection_model60.pth'")
    return model

# =============================================================================
# 8. EVALUATION FUNCTION
# =============================================================================
def evaluate_model(model, data_dir, actions, sequence_length=60, batch_size=32):
    model.eval()
    dataset = SignLanguageVideoDataset(data_dir, actions, sequence_length, augment=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=actions)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    return accuracy, cm, cr

# =============================================================================
# 9. LLM INTEGRATION: GLOSS-TO-SENTENCE TRANSLATION USING LLAMA-3.2-1B
# =============================================================================
# Ensure your transformers version is updated: pip install --upgrade transformers
model_id = "meta-llama/Llama-3.2-1B"
HF_TOKEN = os.environ.get("HF_TOKEN")
llm_generator = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=HF_TOKEN,
    trust_remote_code=True
)

prompt_template = (
    "Convert the following gloss words into a complete, natural English sentence.\n"
    "Gloss words are simplified and may be missing small words like 'am', 'is', 'to', or correct word order.\n"
    "Just return the corrected sentence, starting with a capital letter and ending with a period or question mark.\n"
    "Gloss: {}\n"
    "Sentence:"
)

def gloss_to_sentence(glosses, generator, prompt_template=prompt_template):
    if len(set(glosses)) < 2:
        return ""
    gloss_string = " ".join(glosses)
    prompt = prompt_template.format(gloss_string)
    result = generator(prompt, max_new_tokens=30, do_sample=True, temperature=0.7)
    sentence = result[0]['generated_text']
    if prompt in sentence:
        sentence = sentence.replace(prompt, "").strip()
    sentence = sentence.split("\n")[0].strip()
    print("LLM generated response : ", sentence)
    return sentence

# =============================================================================
# 10. REAL-TIME SLIDING WINDOW PREDICTION WITH CONFIDENCE THRESHOLDING AND LLM INTEGRATION
# =============================================================================
executor = concurrent.futures.ThreadPoolExecutor(max_workers=1)
llm_future = None

def sliding_window_prediction(video_file, model, actions, window_size=60, smoothing_window=20, threshold=0.7, gap_frames=20):
    colors = [(245,117,16), (117,245,16), (16,117,245), (0,255,0)]
    sequence = []  # Current sliding window of keypoints
    predictions = []  # Recent predicted class indices
    prediction_confidences = []  # Corresponding confidence scores
    displayed_prediction = "NA"
    accumulated_glosses = []  # Accumulated distinct glosses for the current sentence
    gap_counter = 0  # Count consecutive frames with "NA"
    sentence_generated = ""
    global llm_future  # Use the global future
    
    cap = cv2.VideoCapture(video_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            keypoints = extract_all_keypoints(results)
            sequence.append(keypoints)
            if len(sequence) > window_size:
                sequence = sequence[-window_size:]
            
            if len(sequence) == window_size:
                seq_array = np.stack(sequence, axis=0)
                seq_tensor = torch.tensor(seq_array, dtype=torch.float32).unsqueeze(0).to(device)
                
                model.eval()
                with torch.no_grad():
                    outputs = model(seq_tensor)
                    prob = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                    pred_idx = np.argmax(prob)
                    conf = prob[pred_idx]
                    predictions.append(pred_idx)
                    prediction_confidences.append(conf)
                
                if len(prediction_confidences) >= smoothing_window:
                    recent_preds = predictions[-smoothing_window:]
                    recent_confs = prediction_confidences[-smoothing_window:]
                    if (len(set(recent_preds)) == 1) and all(c >= threshold for c in recent_confs):
                        current_prediction = actions[pred_idx]
                    else:
                        current_prediction = "NA"
                    
                    if current_prediction != displayed_prediction:
                        displayed_prediction = current_prediction
                        
                    if displayed_prediction != "NA":
                        if not accumulated_glosses or accumulated_glosses[-1] != displayed_prediction:
                            accumulated_glosses.append(displayed_prediction)
                        gap_counter = 0
                    else:
                        gap_counter += 1
                        
                    # Only trigger LLM generation if we have at least 2 distinct glosses
                    # and if no generation is already in progress.
                    if gap_counter >= gap_frames and len(set(accumulated_glosses)) >= 2:
                        if llm_future is None or llm_future.done():
                            print("Glosses sent to LLM for parsing:", accumulated_glosses)
                            llm_future = executor.submit(gloss_to_sentence, accumulated_glosses, llm_generator)
                            accumulated_glosses = []
                            gap_counter = 0
                    if llm_future is not None and llm_future.done():
                        sentence_generated = llm_future.result()
                
                vis_frame = prob_viz(prob, actions, image, colors)
                cv2.rectangle(vis_frame, (0, 0), (640, 40), (245,117,16), -1)
                cv2.putText(vis_frame, displayed_prediction, (3, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                if sentence_generated:
                    cv2.putText(vis_frame, sentence_generated, (3, 70),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Sliding Window Action Detection", vis_frame)
            else:
                cv2.imshow("Sliding Window Action Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# =============================================================================
# 11. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    actions = [
    "Baby", "Bill", "Boy", "Brother", "Daughter", "Death",
    "Doctor", "Family", "Father", "Girl", "he", "healthy",
    "Hospital", "Husband", "I", "it", "Man", "Medicine",
    "Money", "Mother", "Parent", "Patient", "Price",
    "she", "sick", "Sign", "Sister", "Son", "Student",
    "they", "we", "Wife", "Woman", "you", "you (plural)"
    ]

    extracted_keypoints_dir = "./extracted_keypoints"
    raw_dataset_dir = "./dataset"
    
    # Uncomment the block below to extract keypoints from raw training videos.
    # for action in actions:
    #     action_raw_folder = os.path.join(raw_dataset_dir, action)
    #     action_extracted_folder = os.path.join(extracted_keypoints_dir, action)
    #     os.makedirs(action_extracted_folder, exist_ok=True)
    #     video_files = [vf for vf in os.listdir(action_raw_folder) if vf.lower().endswith((".mp4", ".avi", ".mov"))]
    #     for video_file in video_files:
    #         video_path = os.path.join(action_raw_folder, video_file)
    #         video_basename = os.path.splitext(video_file)[0]
    #         video_output_folder = os.path.join(action_extracted_folder, video_basename)
    #         os.makedirs(video_output_folder, exist_ok=True)
    #         print(f"Processing video: {video_path}")
    #         process_video_and_save_frames(video_path, video_output_folder, target_frames=60)
    
    input_size = 1662
    num_classes = len(actions)
    model = ActionModelPyTorch(input_size=input_size, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load("action_detection_model60.pth", map_location=device))
    
    test_video_file = "./videos/SourikSampleVideo.mp4"  # Adjust path as needed
    sliding_window_prediction(test_video_file, model, actions, window_size=60, smoothing_window=20, threshold=0.7)


Device set to use cuda:0
libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast

Glosses sent to LLM for parsing: ['I', 'Sick']
LLM generated response :  I am sick.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Glosses sent to LLM for parsing: ['Sick', 'Price']
LLM generated response :  The sick price for this product is £6.00.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Glosses sent to LLM for parsing: ['I', 'Sick']
LLM generated response :  I am sick.
